In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
from os import getcwd
from os.path import join
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, join(getcwd(), "../module_code"))

import data.load
import cli_utils 
from data.utils import loading_message

sys.argv = [sys.argv[0]]
cli_utils.load_cli_args("../options.yml")
args = cli_utils.init_cli_args()

args.cedars_crrt_data_dir = "C:/Users/jeffe/OneDrive - UCLA IT Services/UCLA/2023_Winter/Rotation/Data/Cedars"

# Outcomes

### Loading Outcomes

In [ ]:
outcomes = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'CRRT Deidentified 2015-2021YTD_VF.xlsx'), sheet_name="2015-2021 YTD")
outcomes_ucla = pd.read_excel(os.path.join(args.ucla_crrt_data_dir,'CRRT Deidentified 2015-2021YTD_VF.xlsx'), sheet_name="2015-2021 YTD")

# print(outcomes[outcomes.duplicated(subset=['PAT_ID','End Date','CRRT Total Days'])])
# ids = outcomes['IP_PATIENT_ID']
# print(outcomes[ids.isin(outcomes[outcomes.duplicated(subset=['IP_PATIENT_ID','End Date','CRRT Total Days'])]['IP_PATIENT_ID'])].sort_values("IP_PATIENT_ID"))

print(outcomes.columns)
print(outcomes_ucla.columns)
print(len(outcomes))
print(len(outcomes_ucla))

outcomes = data.load.load_outcomes(args.cedars_crrt_data_dir, ["IP_PATIENT_ID", "Start Date"])
outcomes_ucla = data.load.load_outcomes(args.ucla_crrt_data_dir, ["IP_PATIENT_ID", "Start Date"])

print(outcomes.columns)
print(outcomes_ucla.columns)
print(len(outcomes))
print(len(outcomes_ucla))

### Outcomes Statistics

In [ ]:
granular_outcome = outcomes[["Recov. renal funct.", "Transitioned to HD", "Comfort Care", "Expired "]].idxmax(axis=1)
binary_outcome = outcomes["recommend_crrt"]

import matplotlib.pyplot as plt
import seaborn as sns
counts = granular_outcome.value_counts()
counts = counts.reindex(["Transitioned to HD", "Comfort Care", "Expired ", "Recov. renal funct.",])

# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pie.html
plt.pie(
    counts,
    # autopct="%1.1f%%",
    # labels=counts.index,
    # autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    # https://seaborn.pydata.org/tutorial/color_palettes.html
    colors=[
        sns.color_palette("mako")[-1],
        sns.color_palette("rocket")[-1],
        sns.color_palette("rocket")[-3],
        sns.color_palette("mako")[-3],
    ]
)
plt.show()

counts = binary_outcome.value_counts()
# plt.pie(counts,  
plt.pie(
    counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    labels=["Recommend CRRT", "Do Not Recommend CRRT", ],
    colors=[
            sns.color_palette("mako")[-3],
            sns.color_palette("rocket")[-3],
    ],
    startangle=300
)
plt.show()


In [ ]:
granular_outcome = outcomes_ucla[["Recov. renal funct.", "Transitioned to HD", "Comfort Care", "Expired "]].idxmax(axis=1)
binary_outcome = outcomes_ucla["recommend_crrt"]

import matplotlib.pyplot as plt
import seaborn as sns
counts = granular_outcome.value_counts()
counts = counts.reindex(["Transitioned to HD", "Comfort Care", "Expired ", "Recov. renal funct.",])

# https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.pie.html
plt.pie(
    counts,
    # autopct="%1.1f%%",
    # labels=counts.index,
    # autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    # https://seaborn.pydata.org/tutorial/color_palettes.html
    colors=[
        sns.color_palette("mako")[-1],
        sns.color_palette("rocket")[-1],
        sns.color_palette("rocket")[-3],
        sns.color_palette("mako")[-3],
    ]
)
plt.show()

counts = binary_outcome.value_counts()
# plt.pie(counts,  
plt.pie(
    counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    labels=["Recommend CRRT", "Do Not Recommend CRRT", ],
    colors=[
            sns.color_palette("mako")[-3],
            sns.color_palette("rocket")[-3],
    ],
    startangle=300
)
plt.show()


In [ ]:
import numpy as np

# Total
crrt_days = outcomes['CRRT Total Days'].value_counts().sort_index()
fig = plt.figure(figsize=(20, 10))
plt.bar(crrt_days.index, height=crrt_days.values)
plt.title('Distribution of CRRT Total Days')
plt.xlabel('CRRT Total Days')
plt.ylabel('Number of Patients')

recommend_days = outcomes[outcomes['recommend_crrt'] == 1]
crrt_days = recommend_days['CRRT Total Days'].value_counts().sort_index()
plt.bar(crrt_days.index, height=crrt_days.values)
plt.legend(["Not recommend", "Recommend"])
plt.show()

print(outcomes['CRRT Total Days'].describe())
print(recommend_days['CRRT Total Days'].describe())

not_recommend_days = outcomes[outcomes['recommend_crrt'] != 1]
print(not_recommend_days['CRRT Total Days'].describe())


In [ ]:
import numpy as np

# Total
crrt_days = outcomes_ucla['CRRT Total Days'].value_counts().sort_index()
fig = plt.figure(figsize=(20, 10))
plt.bar(crrt_days.index, height=crrt_days.values)
plt.title('Distribution of CRRT Total Days')
plt.xlabel('CRRT Total Days')
plt.ylabel('Number of Patients')

recommend_days = outcomes_ucla[outcomes_ucla['recommend_crrt'] == 1]
crrt_days = recommend_days['CRRT Total Days'].value_counts().sort_index()
plt.bar(crrt_days.index, height=crrt_days.values)
plt.legend(["Not recommend", "Recommend"])
plt.show()

print(outcomes_ucla['CRRT Total Days'].describe())
print(recommend_days['CRRT Total Days'].describe())

not_recommend_days = outcomes_ucla[outcomes_ucla['recommend_crrt'] != 1]
print(not_recommend_days['CRRT Total Days'].describe())

In [ ]:
n_unique = len(outcomes.index.get_level_values('IP_PATIENT_ID').unique())
print('Number of Unique Patients: ', n_unique)

repeats_per_patient = outcomes.index.get_level_values('IP_PATIENT_ID').value_counts().values
fig = plt.figure(figsize=(10,6))
counts, edges, bars = plt.hist(repeats_per_patient, bins=range(min(repeats_per_patient), max(repeats_per_patient) + 2, 1), align='left')
plt.xticks(range(min(repeats_per_patient), max(repeats_per_patient) + 2, 1))
plt.bar_label(bars)
plt.title('Histogram of Patients with Multiple CRRTs')
plt.xlabel('Number of Treatments')
plt.ylabel('Number of Patients')
plt.show()

# for i in range(min(repeats_per_patient)+1, max(repeats_per_patient)+1):
#     n_treatments = outcomes.index.get_level_values('IP_PATIENT_ID').value_counts()
#     n_treatments = n_treatments[n_treatments==i].index
#     recommend_days = outcomes[outcomes.index.get_level_values('IP_PATIENT_ID').isin(n_treatments)].reset_index()
#     recommend_days = recommend_days.set_index(['IP_PATIENT_ID','Num Prev CRRT Treatments'])
#     fig = plt.figure(figsize=(10,6))
#     recommend_days.unstack('IP_PATIENT_ID')['recommend_crrt'].plot.line()
#     plt.legend([])
#     plt.show()

for i in range(min(repeats_per_patient), max(repeats_per_patient)+1):
    print(f'Patients with {i} treatments')
    n_treatments = outcomes.index.get_level_values('IP_PATIENT_ID').value_counts()
    n_treatments = n_treatments[n_treatments==i].index
    
    recommend_days = outcomes[outcomes.index.get_level_values('IP_PATIENT_ID').isin(n_treatments)]

    fig = plt.figure(figsize=(20,6))
    for j in range(i):
        plt.subplot(1, i+1, j+1)
        temp = recommend_days[recommend_days['Num Prev CRRT Treatments'] == j]['recommend_crrt']
        counts, edges, bars = plt.hist(temp, bins=[0,1,2], align='left')
        plt.xticks([0,1])
        plt.bar_label(bars)
        plt.gca().set_title(f'After Treatment {j+1}')

    plt.show()


In [ ]:
years = outcomes['CRRT Year']
fig = plt.figure(figsize=(10,6))
counts, edges, bars = plt.hist(years, bins=range(min(years), max(years) + 2, 1), align='left')
plt.xticks(range(min(years), max(years) + 1, 1))
plt.bar_label(bars)
plt.title('Histogram of Treatment Years')
plt.xlabel('Year')
plt.ylabel('Number of Patients')

recommend_days = outcomes[outcomes['recommend_crrt'] == 1]
recommend_years = recommend_days['CRRT Year']
counts, edges, bars = plt.hist(recommend_years, bins=range(min(years), max(years) + 2, 1), align='left')
plt.bar_label(bars)
plt.legend(["Not recommend", "Recommend"])
plt.show()

print(f"min date: {outcomes['End Date'].min()}")
print(f"max date: {outcomes['End Date'].max()}")


# UCLA to Cedars

## File Descriptions

In [ ]:
cedars_features = [
    "karumanchi_00001867_adt - anonymized.xlsx", # admission, discharge, transfer
    "karumanchi_00001867_allergy - anonymized.xlsx",
    "karumanchi_00001867_demographics - anonymized.xlsx",
    "karumanchi_00001867_enc_dx - anonymized.xlsx",
    "karumanchi_00001867_encounters - anonymized.xlsx",
    "karumanchi_00001867_family_hx - anonymized.xlsx",
    "karumanchi_00001867_flowsheet_daily_io - anonymized.xlsx", # have to resave as xlsx
    "karumanchi_00001867_flowsheet_vitals - anonymized.xlsx", # have to resave as xlsx
    "karumanchi_00001867_labs - anonymized.xlsx", 
    "karumanchi_00001867_medications - anonymized.xlsx",
    "karumanchi_00001867_problem_list - anonymized.xlsx",
    "karumanchi_00001867_procedures - anonymized.xlsx", # have to resave as xlsx
    "karumanchi_00001867_social_hx - anonymized (1).xlsx",
]

ucla_features = [
    'Allergies.txt',
    'Encounter_Diagnoses.txt',
    'Encounters.txt',
    'Family_History.txt',
    'Flowsheet_Vitals.txt',
    'Hospital_Unit_Transfers.txt',
    'Labs.txt',
    'Medications.txt',
    'Patient_Demographics.txt',
    'Problem_List_Diagnoses.txt',
    'Problem_Lists.txt',
    'Procedures.txt',
    'Providers.txt',
    'Social_History.txt'
]

mapping = {
    "karumanchi_00001867_adt - anonymized.xlsx": 'Hospital_Unit_Transfers.txt',
    "karumanchi_00001867_allergy - anonymized.xlsx": 'Allergies.txt',
    "karumanchi_00001867_demographics - anonymized.xlsx": 'Patient_Demographics.txt',
    "karumanchi_00001867_enc_dx - anonymized.xlsx": 'Encounter_Diagnoses.txt',
    "karumanchi_00001867_encounters - anonymized.xlsx": 'Encounters.txt',
    "karumanchi_00001867_family_hx - anonymized.xlsx": 'Family_History.txt',
    "karumanchi_00001867_flowsheet_daily_io - anonymized.xlsx": None,
    "karumanchi_00001867_flowsheet_vitals - anonymized.xlsx": 'Flowsheet_Vitals.txt',
    "karumanchi_00001867_labs - anonymized.xlsx": 'Labs.txt',
    "karumanchi_00001867_medications - anonymized.xlsx": 'Medications.txt',
    "karumanchi_00001867_problem_list - anonymized.xlsx": 'Problem_Lists.txt', #'Problem_List_Diagnoses.txt'
    "karumanchi_00001867_procedures - anonymized.xlsx": 'Procedures.txt',
    "karumanchi_00001867_social_hx - anonymized (1).xlsx": 'Social_History.txt',
} # Missing from cedars: Providers

# UCLA ignore
ucla_ignore = [
    'Patient_Identifiers.txt',
]

# Used in UCLA
FILE_NAMES = {
    "cpt": "Procedures.txt",
    "pr": "Problem_Lists.txt",
    "pr_dx": "Problem_List_Diagnoses.txt",
    "labs": "Labs.txt",
    "rx": "Medications.txt",
    "vitals": "Flowsheet_Vitals.txt",
    "dx": "Encounter_Diagnoses.txt",
}

## Cedars Counts

In [ ]:
from data.utils import read_files_and_combine

enc = pd.read_csv(os.path.join(args.cedars_crrt_data_dir, 'Encounters.txt'))

for file in mapping.keys():
    # df = read_files_and_combine([file], args.cedars_crrt_data_dir, on=["IP_PATIENT_ID"])

    if mapping[file] is None:
        continue

    df = pd.read_csv(os.path.join(args.cedars_crrt_data_dir, mapping[file]))
    # df2 = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, file))
    # df2.rename(columns={'PAT_ID': "IP_PATIENT_ID", 'PAT_ENC_CSN_ID':'IP_ENCOUNTER_ID'}, inplace=True)
    # assert all(x == y for x, y in zip(df.columns, df2.columns)), (df.columns, df2.columns)
    # for column in df.columns:
    #     if 'Month' in column or 'time' in column.lower() or 'date' in column.lower():
    #         continue
    #     set1 = set(df[column].unique())
    #     set2 = set(df2[column].unique())
    #     print(column, set1.difference(set2))

    if 'labs' in file:
        df = df.merge(enc[['IP_ENCOUNTER_ID', 'IP_PATIENT_ID']], on='IP_ENCOUNTER_ID', how='left')
        # coalesce cost column to get first non NA value
        df['IP_PATIENT_ID'] = df['IP_PATIENT_ID_x'].combine_first(df['IP_PATIENT_ID_y'])
        # remove the cols
        df = df.drop(['IP_PATIENT_ID_x', 'IP_PATIENT_ID_y'], 1)
        
    merged = pd.merge(df, outcomes, on="IP_PATIENT_ID", how="inner")
    
    print("*"*25 +  f" {file.split('_')[2]} " + "*" * 25)
    print(f"N null: {df['IP_PATIENT_ID'].isna().sum()}, N entries: {len(df)}, N unique IDs: {len(df.drop_duplicates('IP_PATIENT_ID'))}")
    print(f"MERGED WITH OUTCOME: N entries {len(merged)}, N unique IDs: {len(merged.drop_duplicates('IP_PATIENT_ID'))}")
    # print(f"Difference in # unique patients: {len(outcomes)  - len(df.drop_duplicates('IP_PATIENT_ID'))}")

## UCLA Counts

In [ ]:
from data.utils import read_files_and_combine

enc = pd.read_csv(os.path.join(args.cedars_crrt_data_dir, 'Encounters.txt'))

for file in mapping.keys():
    # df = read_files_and_combine([file], args.cedars_crrt_data_dir, on=["IP_PATIENT_ID"])

    if mapping[file] is None:
        continue

    try:
        df = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, mapping[file]))
    except:
        df = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, mapping[file]), encoding='cp1252')
    
    df.columns = [col.upper() for col in df.columns]

    # if 'labs' in file:
    #     df = pd.merge(df, enc[['IP_ENCOUNTER_ID', 'IP_PATIENT_ID']], on='IP_ENCOUNTER_ID', how='left')
    #     # coalesce cost column to get first non NA value
    #     df['IP_PATIENT_ID'] = df['IP_PATIENT_ID_x'].combine_first(df['IP_PATIENT_ID_y'])
    #     # remove the cols
    #     df = df.drop(['IP_PATIENT_ID_x', 'IP_PATIENT_ID_y'], 1)
        
    merged = pd.merge(df, outcomes_ucla, on="IP_PATIENT_ID", how="inner")
    
    print("*"*25 +  f" {file.split('_')[2]} " + "*" * 25)
    print(f"N null: {df['IP_PATIENT_ID'].isna().sum()}, N entries: {len(df)}, N unique IDs: {len(df.drop_duplicates('IP_PATIENT_ID'))}")
    print(f"MERGED WITH OUTCOME: N entries {len(merged)}, N unique IDs: {len(merged.drop_duplicates('IP_PATIENT_ID'))}")
    # print(f"Difference in # unique patients: {len(outcomes)  - len(df.drop_duplicates('IP_PATIENT_ID'))}")

# Comparisons of features (i.e., DataFrame Columns)

## Hospital Transfers

In [ ]:
adt = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_adt - anonymized.xlsx'))
hospital_transfer = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Hospital_Unit_Transfers.txt'))

In [ ]:
print(adt.columns)
print(hospital_transfer.columns)

#### Notes
* Same columns under different names
* We don't use these futures, just keeping here for reference

## Allergies

In [ ]:
cedars_allerg = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_allergy - anonymized.xlsx'))
ucla_allerg = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Allergies.txt'))

In [ ]:
print(cedars_allerg.columns)
print(ucla_allerg.columns)

## Demographics

In [ ]:
cedars_demog = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_demographics - anonymized.xlsx'))
ucla_demog = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Patient_Demographics.txt'))

In [ ]:
print(cedars_demog.columns)
print(ucla_demog.columns)

In [ ]:
print(cedars_demog.columns)
print(ucla_demog.columns)

In [ ]:
cedars_demog_clean = cedars_demog.rename(
            {
                # CONTROLS
                "GENDER": "SEX",
                "IP_CURRENT_PCP_ID": "PCP_IP_PROVIDER_ID",
                "VITAL_STATUS": "KNOWN_DECEASED",

                # CEDARS
                'CURRENT_AGE': "AGE",
                'RACE_1': 'RACE',
                'ETHNIC_GROUP': 'ETHNICITY',
                'LIVING_STATUS': 'KNOWN_DECEASED',
                "CURRENT_PCP_ID": "PCP_IP_PROVIDER_ID",
                'PAT_ID': "IP_PATIENT_ID"
            },
            axis=1,
        )

# Cleanup RACE for Cedars data. Combine RACE_2 and RACE_3 into multiple races
if 'RACE_2' in cedars_demog_clean.columns and 'RACE_3' in cedars_demog_clean.columns:
    cedars_demog_clean.loc[cedars_demog_clean['RACE_2'].notna() | cedars_demog_clean['RACE_3'].notna(), 'RACE'] = 'Multiple Races'
    cedars_demog_clean = cedars_demog_clean.replace({'RACE': {'White': 'White or Caucasian', 'Patient Declined': 'Patient Refused'}}) 
    cedars_demog_clean = cedars_demog_clean.drop(["RACE_2","RACE_3"], axis=1)


In [ ]:
age = cedars_demog['CURRENT_AGE']
plt.figure(figsize=(20,10))
counts, edges, bars = plt.hist(age, bins=range(0, max(age) + 5, 5), align='mid')
plt.bar_label(bars)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.show()

In [ ]:
import seaborn as sns
sex = cedars_demog['SEX']
plt.figure(figsize=(5,5))
counts = sex.value_counts()
plt.pie(
    counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    labels=["Male", "Female", "Unknown" ],
    colors=[
            sns.color_palette("mako")[-3],
            sns.color_palette("rocket")[-3],
            sns.color_palette("rocket")[-2],
    ],
    startangle=300
)
plt.show()

In [ ]:
import seaborn as sns
race = cedars_demog['RACE_1']
plt.figure(figsize=(7,7))
counts = race.value_counts()
print(counts)
plt.pie(
    counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    labels=list(counts.index),
    startangle=300
)
plt.show()

race = cedars_demog_clean['RACE']
plt.figure(figsize=(7,7))
counts = race.value_counts()
print(counts)
plt.pie(
    counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    labels=list(counts.index),
    startangle=300
)
plt.show()


race = cedars_demog['RACE_2']
plt.figure(figsize=(7,7))
counts = race.value_counts()
print(counts)
plt.pie(
    counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    labels=list(counts.index),
    startangle=300
)
plt.show()

race = cedars_demog['RACE_3']
plt.figure(figsize=(7,7))
counts = race.value_counts()
print(counts)
plt.pie(
    counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    labels=list(counts.index),
    startangle=300
)
plt.show()



In [ ]:
import seaborn as sns
ethnicity = cedars_demog_clean['ETHNICITY']
plt.figure(figsize=(5,5))
counts = ethnicity.value_counts()
print(counts)
plt.pie(
    counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    labels=list(counts.index),
    startangle=300
)
plt.show()

In [ ]:
import seaborn as sns
ethnicity = cedars_demog['LIVING_STATUS']
plt.figure(figsize=(5,5))
counts = ethnicity.value_counts()
print(counts)
plt.pie(
    counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(counts)/100),
    labels=list(counts.index),
    startangle=300
)
plt.show()

#### Notes
* RACE_3 is basically empty, don't use
* RACE_2 is also almost empty - consists of few race descriptions, mostly 'Other' or 'Empty'.
* Probaly always take RACE_1. Or have to create a new 'Mixed' column
* Some descriptions are different (e.g. Cedars has "White" but UCLA has "White or Caucausian")
* Cedars has "Alive/Deceased" but UCLA has "Known Deceased/Not Known Deceased"
* Age is int not float

## Encounters

In [ ]:
cedars_enc = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_encounters - anonymized.xlsx'))
ucla_enc = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Encounters.txt'), encoding="cp1252")

In [ ]:
print(cedars_enc.columns)
print(ucla_enc.columns)

## Encounters Diagnosis

In [ ]:
cedars_enc_dx = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_enc_dx - anonymized.xlsx'))
ucla_enc_dx = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Encounter_Diagnoses.txt'))

In [ ]:
print(cedars_enc_dx.columns)
print(ucla_enc_dx.columns)

In [ ]:
print(cedars_enc_dx.groupby("CURRENT_ICD10_LIST").size().sort_values(ascending=False)[:15])
icd_dx_dates = pd.to_datetime(cedars_enc_dx["CONTACT_DATE"])
print(f"min date: {icd_dx_dates.min()}, max_date: {icd_dx_dates.max()}")

#### Notes
* UCLA has ICD_TYPE. Cedars it's all ICD10

left (cedars), right (ucla)
* CONACT_DATE == DIAGNOSIS_DATE
* CURRENT_ICD10_LIST == ICD_CODE
* DX_NAME == ICD_DESC
* PRIMARY_DX_YN == PRIMARY_DIAGNOSIS_FLAG (Y/N ==> S/P)
* HSP_FINAL_DIAGNOSIS_FLAG == HSP_FINAL_DIAGNOSIS_FLAG (Y/N ==> 0.0/1.0)
* PRESENT_ON_ADM == PRESENT_ON_ADMIT_FLAG (Y/N ==> 0.0/1.0)

* Cedars has no 'ADMIT_DIAGNOSIS_FLAG'
* Cedars has extra 'RECORD_TYPE'

## Family History

In [ ]:
cedars_fam = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_family_hx - anonymized.xlsx'))
ucla_fam = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Family_History.txt'))

In [ ]:
print(cedars_fam.columns)
print(ucla_fam.columns)

## Flowsheet

In [ ]:
ucla_flow = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Flowsheet_Vitals.txt'))
cedars_flow = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_flowsheet_vitals - anonymized.xlsx'))

In [ ]:
cedars_flow_daily = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_flowsheet_daily_io - anonymized.xlsx'))

In [ ]:
# print(cedars_flow_daily.columns)
print(ucla_flow.columns)
print(cedars_flow.columns)

#### Notes
* UCLA doesn't have the equivalent 'daily_io'
* UCLA has 'VITAL_SIGN_SOURCE' which is all just 'FLOWSHEET'
* VITAL_SIGN_TYPE == MEAS_NAME (Cedars - ALL CAPS)
* VITAL_SIGN_VALUE == MEAS_VALUE
* VITAL_SIGN_TAKEN_TIME == RECORDED_TIME

* Cedars BP maybe needs to be separated into SBP and DBP

In [ ]:
cedars_flow["MEAS_NAME"].replace({"BP": "SBP/DBP"}, inplace=True)
cedars_flow_drop = cedars_flow.dropna(subset=['MEAS_VALUE'])
explode_cols = ["MEAS_VALUE", "MEAS_NAME"]
cedars_flow_drop['MEAS_VALUE'] = cedars_flow_drop['MEAS_VALUE'].astype(str)

# Ref: https://stackoverflow.com/a/57122617/1888794
# don't explode the columsn you set index to, explode the rest via apply, reset everything to normal
cedars_flow_splitbp = cedars_flow_drop.apply(
        lambda col: col.str.split("/") if col.name in explode_cols else col
    ).explode(explode_cols)

In [ ]:
cedars_flow_splitbp['MEAS_NAME'].unique()

## Labs

In [ ]:
cedars_labs = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_labs - anonymized.xlsx'))
ucla_labs = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Labs.txt'),  encoding="cp1252")

In [ ]:
print(cedars_labs.columns)
print(ucla_labs.columns)

In [ ]:
print(ucla_labs['IP_PATIENT_ID'].isnull().values.any())
print(cedars_labs['PAT_ID'].isnull().values.any())

#### Notes
* Same except Cedars "NAME" is UCLA "COMPONENT_NAME"
* Lots of missing PAT_ID in cedars - use enc_id?

## Medications

In [ ]:
cedars_meds = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_medications - anonymized.xlsx'))
ucla_meds = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Medications.txt'))

In [ ]:
print(cedars_meds.columns)
print(ucla_meds.columns)

#### Notes
CEDARS ==> UCLA
* NAME == MEDICATION_NAME
* TAKEN_TIME == TAKEN_TIME_ORDER_DATE
* PHARM_CLASS == MEDISPAN_CLASS_NAME
* ORDERING_DATE == ORDER_DATE

* CEDARS has ROUTE

## Problem List

In [ ]:
cedars_problems = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_problem_list - anonymized.xlsx'))
cedars_problems_updated = pd.read_excel(os.path.join(args.cedars_crrt_data_dir,'karumanchi_00001867_problem_list - anonymized - updated.xlsx'))
ucla_problems = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Problem_Lists.txt'),encoding="cp1252")
ucla_problems_dx = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Problem_List_Diagnoses.txt'))

In [ ]:
display(cedars_problems)
display(cedars_problems[cedars_problems['NOTED_DATE'].isna()])
display(cedars_problems[~cedars_problems['DATE_OF_ENTRY'].isna()]['STATUS'].value_counts())

In [ ]:
display(ucla_problems)
display(ucla_problems[ucla_problems['noted_date'].isna()])
display(ucla_problems[ucla_problems['date_of_entry'].isna()])

In [ ]:
display(cedars_problems_updated)
display(cedars_problems_updated[cedars_problems_updated['NOTED_DATE'].isna()])
display(cedars_problems_updated[~cedars_problems_updated['DATE_OF_ENTRY'].isna()]['STATUS'].value_counts())

In [ ]:
print(cedars_problems.columns)
print(cedars_problems_updated.columns)
print(ucla_problems.columns)
print(ucla_problems_dx.columns)

In [ ]:
from hcuppy.ccs import CCSEngine
ce = CCSEngine(mode="dx")
ccs_dict = cedars_problems[cedars_problems['STATUS']=='ACTIVE']['CURRENT_ICD10_LIST'].apply(lambda icd_code: ce.get_ccs(icd_code))
# series of dicts, explode each dict attribute to its own column
ccs_dict = pd.DataFrame(ccs_dict.values.tolist())
ccs_dict.columns = ["CCS_CODE", "CCS_DESCRIPTION", "CCS_LEVEL1", "CCS_LEVEL1_DESCRIPTION", "CCS_LEVEL2", "CCS_LEVEL2_DESCRIPTION"]

# combine the granular icd codes with the higher level CCS ones
cedars_problems_ccs = pd.concat([cedars_problems, ccs_dict], axis=1)

print(cedars_problems_ccs.groupby("CCS_CODE").size().sort_values(ascending=False)[:15])
print(len(cedars_problems_ccs['CCS_CODE'].unique()))
onehot_ccs = pd.get_dummies(cedars_problems_ccs[["PAT_ID", "CCS_CODE"]], columns=["CCS_CODE"])
onehot_ccs

In [ ]:
from hcuppy.ccs import CCSEngine
ce = CCSEngine(mode="dx")
ccs_dict = cedars_problems_updated[cedars_problems_updated['STATUS']=='ACTIVE']['CURRENT_ICD10_LIST'].apply(lambda icd_code: ce.get_ccs(icd_code))
# series of dicts, explode each dict attribute to its own column
ccs_dict = pd.DataFrame(ccs_dict.values.tolist())
ccs_dict.columns = ["CCS_CODE", "CCS_DESCRIPTION", "CCS_LEVEL1", "CCS_LEVEL1_DESCRIPTION", "CCS_LEVEL2", "CCS_LEVEL2_DESCRIPTION"]

# combine the granular icd codes with the higher level CCS ones
cedars_problems_ccs = pd.concat([cedars_problems_updated, ccs_dict], axis=1)

print(cedars_problems_ccs.groupby("CCS_CODE").size().sort_values(ascending=False)[:15])
print(len(cedars_problems_ccs['CCS_CODE'].unique()))

onehot_ccs = pd.get_dummies(cedars_problems_ccs[["PAT_ID", "CCS_CODE"]], columns=["CCS_CODE"])
onehot_ccs

#### Notes

* CEDARS problems combines problem list and problem list dx


* CONTACT_DATE => encounter_date
* DESCRIPTION => prob_desc
* STATUS => problem_status





* UCLA dx has 'diagnosis source; which is just 'PROBLEM_LIST'
* UCLA has ICD_TYPE. Cedars it's all ICD10
* DX_NAME => icd_desc
* Cedars can have multiple codes??? 
* Cedars has ACCT_DC_YN
* Cedars has PRIOTITY
* UCLA has diagnosis_date


## Procedures

In [ ]:
cedars_proc = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_procedures - anonymized.xlsx'))
ucla_proc = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Procedures.txt'))

In [ ]:
print(cedars_proc.columns)
print(ucla_proc.columns)

#### Notes
* PROC_START_TIME is more granular than PROC_DATE
* Weird proc codes

## Social History

In [ ]:
cedars_soc = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_social_hx - anonymized (1).xlsx'))
ucla_soc = pd.read_csv(os.path.join(args.ucla_crrt_data_dir, 'Social_History.txt'))

In [ ]:
print(cedars_soc.columns)
print(ucla_soc.columns)

#### Notes
* SMOKING_TOBACCO_USE => SMOKING_TOB_STATUS
* TOBACCO_USE => TOBACCO_USER

* UCLA has CIGARETTES_YN

* ALCOHOL_USE => ALCOHOL_USER
* ALCOHOL_COMMENT => ALCOHOL_TYPE (MIGHT NEED FORMATTING)

* Cedars has ILLICIT_DRUG_USER

# Using Load Functions

## Static Features

In [ ]:
import data.load
import data.preprocess

static_df = data.load.load_static_features(args.cedars_crrt_data_dir).set_index("IP_PATIENT_ID")

# ids = static_df['PAT_ID']
# dupe = static_df[ids.isin(ids[ids.duplicated()])].sort_values("PAT_ID")
# print(dupe)

temp_df = outcomes.reset_index().set_index('IP_PATIENT_ID')
features_with_outcomes = static_df.join(temp_df, how="inner")

display(static_df)


## Longitudinal Features

In [ ]:
from data.longitudinal_utils import (
    get_time_window_mask
)

import data.longitudinal_features as lf

from typing import Union

temp = outcomes.copy()
temp.index.rename({'PAT_ID':'IP_PATIENT_ID'}, inplace=True)
window = get_time_window_mask(temp, {'YEARS': 10, 'MONTHS': 0, 'DAYS': 7}, None, 'End Date', 0)
ucla_window = get_time_window_mask(outcomes_ucla.copy(), {'YEARS': 10, 'MONTHS': 0, 'DAYS': 7}, None, 'End Date', 0)

### Diagnoses

In [ ]:
diagnoses = lf.load_diagnoses(args.cedars_crrt_data_dir,
                        time_window = window, time_interval='1D')
display(diagnoses)

### Vitals

In [ ]:
vitals = lf.load_vitals(args.cedars_crrt_data_dir,
                        time_window = window, time_interval='1D')
display(vitals)

In [ ]:
display(vitals)

### Medication

In [ ]:
meds = lf.load_medications(args.cedars_crrt_data_dir, 
                        time_window = window, time_interval=None)

In [ ]:
display(meds)

### Labs

In [ ]:
labs = lf.load_labs(args.cedars_crrt_data_dir, 
                        time_window = window, time_interval=None)
display(labs)

In [ ]:
display(labs)

### Problems

In [ ]:
problems = lf.load_problems(args.cedars_crrt_data_dir,  
                        time_window = window, time_interval=None)
display(problems)

In [ ]:
display(problems)

### Procedures

In [ ]:
proc = lf.load_procedures(args.cedars_crrt_data_dir, 
                        time_window = window, time_interval=None)
display(proc)

In [ ]:
display(proc)

# Value Alignment

In [ ]:
import jellyfish
import itertools
from fuzzywuzzy import fuzz

def jelly_pairwise_diff(left:list, right:list) -> pd.DataFrame:
    results = {
        combo: jellyfish.levenshtein_distance(*combo) for combo in itertools.product(left, right)
    }
    distances = pd.DataFrame({'Distance':results.values()}, index = results.keys()).sort_values('Distance')
    return distances

def fuzzy_pairwise_diff(left:list, right:list, mode:str = None) -> pd.DataFrame:

    if mode == 'sort':
        comparison_function = fuzz.token_sort_ratio
    elif mode == 'partial':
        comparison_function = fuzz.partial_ratio
    else:
        comparison_function = fuzz.ratio
    
    results = {
        combo: comparison_function(*combo) for combo in itertools.product(left, right)
    }
    distances = pd.DataFrame({'Distance':results.values()}, index = results.keys()).sort_values('Distance', ascending=False)
    return distances

def assess_value_alignment(df1, df2, cols=None, len_only=False, permute=False):

    if cols is None:
        cols = set(df1.columns).intersection(set(df2.columns))
    else:
        cols = set(cols)

    cols = cols - set({'IP_PATIENT_ID', 'IP_ENCOUNTER_ID'})
    
    if not permute:
        for column in cols:
            unique1 = set(df1[column].unique())
            unique2 = set(df2[column].unique())

            print('Assessing column: ', column)
            if len_only:
                print("Only in first df: ", len(unique1.difference(unique2)))
                print("Only in second df: ", len(unique2.difference(unique1)))
                print("In both df: ", len(unique2.intersection(unique1)))
            else:
                print("Only in first df: ", unique1.difference(unique2))
                print("Only in second df: ", unique2.difference(unique1))
                print("In both df: ", unique2.intersection(unique1))
            print('')
    else:
        for column1 in cols:
            for column2 in cols:
                unique1 = set(df1[column1].unique())
                unique2 = set(df2[column2].unique())

                print('Assessing column: ', column1, ' vs. ', column2)
                if len_only:
                    print("Only in first df: ", len(unique1.difference(unique2)))
                    print("Only in second df: ", len(unique2.difference(unique1)))
                    print("In both df: ", len(unique2.intersection(unique1)))
                else:
                    print("Only in first df: ", unique1.difference(unique2))
                    print("Only in second df: ", unique2.difference(unique1))
                    print("In both df: ", unique2.intersection(unique1))
                print('')

### Demographics

In [ ]:
import data.load
import numpy as np
from pandas import to_numeric

cedars_static_df = data.load.read_files_and_combine(["Patient_Demographics.txt"], args.cedars_crrt_data_dir, how="outer")
ucla_static_df = data.load.read_files_and_combine(["Patient_Demographics.txt"], args.ucla_crrt_data_dir, how="outer")

def static_preprocess(static_df):
    collapse_ethnicity_map = {  # Collapse everything to (Not) Hisp/Lat
            "Not Hispanic or Latino": [
                # UCLA
                "Choose Not to Answer",
                "Patient Refused",
                "Unknown",
                # Cedars
                "Non-Hispanic",
                "Patient Declined",
                np.nan
            ],
            "Hispanic or Latino": [
                # UCLA
                "Mexican, Mexican American, Chicano/a",
                "Hispanic/Spanish origin Other",
                "Puerto Rican",
                "Cuban",
                # Cedars
                "Hispanic",
            ],
        }
    
    
    column_alignment = {
            # CONTROLS
            "GENDER": "SEX",
            "IP_CURRENT_PCP_ID": "PCP_IP_PROVIDER_ID",
            "VITAL_STATUS": "KNOWN_DECEASED",
            # CEDARS
            "CURRENT_AGE": "AGE",
            "RACE_1": "RACE",
            "ETHNIC_GROUP": "ETHNICITY",
            "LIVING_STATUS": "KNOWN_DECEASED",
            "CURRENT_PCP_ID": "PCP_IP_PROVIDER_ID",
        }
    
    static_df = static_df.rename(column_alignment, axis=1).replace(
            {
                "ETHNICITY": {
                    col: collapsed_name
                    for collapsed_name, cols in collapse_ethnicity_map.items()
                    for col in cols
                }
            }
        )

    #### ADD TO LOAD.PY ####
    # Cleanup RACE for Cedars data. Combine RACE_2 and RACE_3 into multiple races
    if "RACE_2" in static_df.columns and "RACE_3" in static_df.columns:
        static_df.loc[
            static_df["RACE_2"].notna() | static_df["RACE_3"].notna(), "RACE"
        ] = "Multiple Races"
        static_df = static_df.replace(
            {
                "RACE": {
                    "White": "White or Caucasian",
                    "Patient Declined": "Patient Refused",
                }
            }
        )
        static_df = static_df.drop(["RACE_2", "RACE_3"], axis=1)

    static_df["AGE"] = static_df["AGE"].astype(float)
    static_df = static_df[
        ~((static_df["SEX"] != "Male") & (static_df["SEX"] != "Female"))
    ]

    collapse_race_map = {
        "Unknown": [
            "Other",
            'Patient Refused',
            np.nan
        ]

    }
    static_df = static_df.replace(
        {
                "RACE": {
                    col: collapsed_name
                    for collapsed_name, cols in collapse_race_map.items()
                    for col in cols
                }
            }
    )
    # static_df = static_df.dropna(subset=["AGE","RACE"])
    #### ADD TO LOAD.PY ####

    static_df = static_df.drop('PCP_IP_PROVIDER_ID', axis=1, errors="ignore")
    # static_df = map_provider_id_to_type(static_df, raw_data_dir)

    # explicitly mapping here instead of numerical encoding automatically so that you know which is which when referencing outputs/data/etc.
    bin_cols_mapping = {
            "SEX": {"Male": 0, "Female": 1},
            "KNOWN_DECEASED": {
                "Not Known Deceased": 0,
                "Known Deceased": 1,
                "Alive": 0,  # Cedars
                "Deceased": 1,  # Cedars
            },
            "ETHNICITY": {
                "Not Hispanic or Latino": 0,
                "Hispanic or Latino": 1,
            },
        }
    static_df = static_df.replace(bin_cols_mapping)
        
    return static_df

cedars_static_df = static_preprocess(cedars_static_df)
ucla_static_df = static_preprocess(ucla_static_df)

assess_value_alignment(cedars_static_df, ucla_static_df)

In [ ]:
cedars_static_df

#### Notes
* Collased Ethnicity to binary categories (hispanic/non-hispanic)
* Renaming of columns to match UCLA
* Cast AGE as float
* Removal of one "unknown" from gender - keep this as unknown
* Combine Race_1, Race_2, Race_3 into "multiple races" if Race_2 or Race_3 are not nan, otherwise take Race_1
* *Drop rows with null ages or race?* - consider null patients as unkown and combine them all with other, unkown, refused, etc. do not remove null age, this will be filtered

In [ ]:
age_cedars = cedars_static_df['AGE']
plt.figure(figsize=(15,10))
counts, edges, bars = plt.hist(age_cedars, bins=range(0, int(max(age_cedars)) + 5, 5), align='mid', alpha=0.5,)
plt.bar_label(bars)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
# plt.show()

age = ucla_static_df['AGE']
# plt.figure(figsize=(20,10))
counts, edges, bars = plt.hist(age, bins=range(0, int(max(age_cedars)) + 5, 5), align='mid', alpha=0.5,)
plt.bar_label(bars)
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Count')
plt.legend(['Cedars','UCLA'])
plt.show()

In [ ]:
plt.figure(figsize=(10,5))

def pie_plot(value_counts, title="Cedars", labels=None):
    if labels is None:
        labels = list(value_counts.index)
    plt.pie(
    value_counts,
    # autopct="%1.1f%%",
    autopct=lambda p : '{:.2f}%  ({:,.0f})'.format(p,p * sum(value_counts)/100),
    labels=labels,
    startangle=300
    )
    plt.title(title)

plt.subplot(1,2,1)
pie_plot(cedars_static_df['SEX'].value_counts(), title="Cedars", labels=['Male', 'Female'])

plt.subplot(1,2,2)
pie_plot(ucla_static_df['SEX'].value_counts(), title="UCLA", labels=['Male', 'Female'])

plt.show()

In [ ]:
plt.figure(figsize=(15,15))

plt.subplot(2,1,1)
pie_plot(cedars_static_df['RACE'].value_counts().reindex(ucla_static_df['RACE'].value_counts().index), title="Cedars")

plt.subplot(2,1,2)
pie_plot(ucla_static_df['RACE'].value_counts(), title="UCLA")

print('-------------------Cedars------------------')
print(cedars_static_df['RACE'].value_counts().reindex(ucla_static_df['RACE'].value_counts().index))
print('-------------------UCLA------------------')
print(ucla_static_df['RACE'].value_counts())

In [ ]:
def stacked_plot(value_count):
    legend = list(value_count.index)
    number = value_count.values

    bottom = 0
    for i in range(len(number)):
        plt.bar('Cedars', number[i], bottom=bottom)
        bottom = number[i] + bottom
        legend[i] = f'{legend[i]}: {number[i]*100:.3f}%'
        # plt.text(0.2, bottom-number[i]/2, number[i])
    plt.legend(legend)

plt.subplot(1,2,1)
stacked_plot(cedars_static_df['RACE'].value_counts(normalize=True))
plt.subplot(1,2,2)
stacked_plot(ucla_static_df['RACE'].value_counts(normalize=True))
plt.show()

* Use side-by-side bar charts

In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(1,2,1)
pie_plot(cedars_static_df['ETHNICITY'].value_counts(), title="Cedars", labels=['Non Hispanic or Latino','Hispanic or Latino'])

plt.subplot(1,2,2)
pie_plot(ucla_static_df['ETHNICITY'].value_counts(), title="UCLA", labels=['Non Hispanic or Latino','Hispanic or Latino'])


In [ ]:
plt.figure(figsize=(10,10))

plt.subplot(1,2,1)
pie_plot(cedars_static_df['KNOWN_DECEASED'].value_counts(), title="Cedars", labels=['Not Known Deceased','Known Deceased'])

plt.subplot(1,2,2)
pie_plot(ucla_static_df['KNOWN_DECEASED'].value_counts(), title="UCLA", labels=['Not Known Deceased','Known Deceased'])

### Diagnoses

In [ ]:
cedars_dx_df = data.load.read_files_and_combine(['Encounter_Diagnoses.txt'], args.cedars_crrt_data_dir)
ucla_dx_df = data.load.read_files_and_combine(['Encounter_Diagnoses.txt'], args.ucla_crrt_data_dir)

# Cedars alignment. Assume contact date is diagnosis date since that's the date we have
cedars_dx_df = cedars_dx_df.rename({"CURRENT_ICD10_LIST": "ICD_CODE",
                        "CONTACT_DATE": "DIAGNOSIS_DATE"}, axis=1)

assess_value_alignment(cedars_dx_df, ucla_dx_df, cols=['ICD_CODE'], len_only=True)

In [ ]:
cedars_diag_icd10 = set(cedars_dx_df['ICD_CODE'])
ucla_diag_icd10 = set(ucla_dx_df['ICD_CODE'])

only_cedars = cedars_diag_icd10.difference(ucla_diag_icd10)
only_ucla = ucla_diag_icd10.difference(cedars_diag_icd10)
both_diag = cedars_diag_icd10.intersection(ucla_diag_icd10)

print('Rows in Cedars Diagnosis File: ', len(cedars_dx_df))
print('Rows in Cedars Diagnosis File with codes unique from UCLA: ', 
      len(cedars_dx_df[cedars_dx_df['ICD_CODE'].isin(only_cedars)]), '|',
      len(cedars_dx_df[cedars_dx_df['ICD_CODE'].isin(only_cedars)])/len(cedars_dx_df))
print('Rows in Cedars Diagnosis File with codes shared with UCLA: ', 
      len(cedars_dx_df[cedars_dx_df['ICD_CODE'].isin(both_diag)]),'|',
      len(cedars_dx_df[cedars_dx_df['ICD_CODE'].isin(both_diag)])/len(cedars_dx_df))

print('Rows in UCLA Diagnosis File: ', len(ucla_dx_df))
print('Rows in UCLA Diagnosis File with codes unique from Cedars: ', 
      len(ucla_dx_df[ucla_dx_df['ICD_CODE'].isin(only_ucla)]),'|',
      len(ucla_dx_df[ucla_dx_df['ICD_CODE'].isin(only_ucla)])/len(ucla_dx_df))
print('Rows in UCLA Diagnosis File with codes shared with Cedars: ', 
      len(ucla_dx_df[ucla_dx_df['ICD_CODE'].isin(both_diag)]),'|',
      len(ucla_dx_df[ucla_dx_df['ICD_CODE'].isin(both_diag)])/len(ucla_dx_df))

# print(cedars_dx_df[cedars_dx_df['ICD_CODE'].isin(only_cedars)]['ICD_CODE'].value_counts()[:10])
# print(cedars_dx_df[cedars_dx_df['ICD_CODE'].isin(both_diag)]['ICD_CODE'].value_counts()[:10])

# print(ucla_dx_df[ucla_dx_df['ICD_CODE'].isin(only_ucla)]['ICD_CODE'].value_counts()[:10])
# print(ucla_dx_df[ucla_dx_df['ICD_CODE'].isin(both_diag)]['ICD_CODE'].value_counts()[:10])

#### Notes
* This one just combine via outer join

### Problems

In [ ]:
cedars_prob = data.load.read_files_and_combine(['Problem_Lists_Outdated.txt'], args.cedars_crrt_data_dir)
ucla_prob = data.load.read_files_and_combine(['Problem_Lists.txt', "Problem_List_Diagnoses.txt"], args.ucla_crrt_data_dir)

print(len(cedars_prob['IP_PATIENT_ID'].unique()))
cedars_prob = cedars_prob.rename({'STATUS': 'PROBLEM_STATUS',
                                    "CURRENT_ICD10_LIST": "ICD_CODE"}, axis=1)
assess_value_alignment(cedars_prob, ucla_prob, cols=['ICD_CODE'], len_only=True)
#### This one just combine via outer join

In [ ]:
cedars_diag_icd10 = set(cedars_prob['ICD_CODE'])
ucla_diag_icd10 = set(ucla_prob['ICD_CODE'])

only_cedars = cedars_diag_icd10.difference(ucla_diag_icd10)
only_ucla = ucla_diag_icd10.difference(cedars_diag_icd10)
both_diag = cedars_diag_icd10.intersection(ucla_diag_icd10)

print('Rows in Cedars Diagnosis File: ', len(cedars_prob))
print('Rows in Cedars Diagnosis File with codes unique from UCLA: ', 
      len(cedars_prob[cedars_prob['ICD_CODE'].isin(only_cedars)]), '|',
      len(cedars_prob[cedars_prob['ICD_CODE'].isin(only_cedars)])/len(cedars_prob))
print('Rows in Cedars Diagnosis File with codes shared with UCLA: ', 
      len(cedars_prob[cedars_prob['ICD_CODE'].isin(both_diag)]),'|',
      len(cedars_prob[cedars_prob['ICD_CODE'].isin(both_diag)])/len(cedars_prob))

print('Rows in UCLA Diagnosis File: ', len(ucla_prob))
print('Rows in UCLA Diagnosis File with codes unique from Cedars: ', 
      len(ucla_prob[ucla_prob['ICD_CODE'].isin(only_ucla)]),'|',
      len(ucla_prob[ucla_prob['ICD_CODE'].isin(only_ucla)])/len(ucla_prob))
print('Rows in UCLA Diagnosis File with codes shared with Cedars: ', 
      len(ucla_prob[ucla_prob['ICD_CODE'].isin(both_diag)]),'|',
      len(ucla_prob[ucla_prob['ICD_CODE'].isin(both_diag)])/len(ucla_prob))

# print(cedars_prob[cedars_prob['ICD_CODE'].isin(only_cedars)]['ICD_CODE'].value_counts()[:10])
# print(cedars_prob[cedars_prob['ICD_CODE'].isin(both_diag)]['ICD_CODE'].value_counts()[:10])

# print(ucla_prob[ucla_prob['ICD_CODE'].isin(only_ucla)]['ICD_CODE'].value_counts()[:10])
# print(ucla_prob[ucla_prob['ICD_CODE'].isin(both_diag)]['ICD_CODE'].value_counts()[:10])

In [ ]:
cedars_prob = data.load.read_files_and_combine(['Problem_Lists.txt'], args.cedars_crrt_data_dir)
ucla_prob = data.load.read_files_and_combine(['Problem_Lists.txt', "Problem_List_Diagnoses.txt"], args.ucla_crrt_data_dir)

cedars_prob = cedars_prob.rename({'STATUS': 'PROBLEM_STATUS',
                                    "CURRENT_ICD10_LIST": "ICD_CODE"}, axis=1)

print(len(cedars_prob['IP_PATIENT_ID'].unique()))
assess_value_alignment(cedars_prob, ucla_prob, cols=['ICD_CODE'], len_only=True)
#### This one just combine via outer join

In [ ]:
cedars_diag_icd10 = set(cedars_prob['ICD_CODE'])
ucla_diag_icd10 = set(ucla_prob['ICD_CODE'])

only_cedars = cedars_diag_icd10.difference(ucla_diag_icd10)
only_ucla = ucla_diag_icd10.difference(cedars_diag_icd10)
both_diag = cedars_diag_icd10.intersection(ucla_diag_icd10)

print('Rows in Cedars Diagnosis File: ', len(cedars_prob))
print('Rows in Cedars Diagnosis File with codes unique from UCLA: ', 
      len(cedars_prob[cedars_prob['ICD_CODE'].isin(only_cedars)]), '|',
      len(cedars_prob[cedars_prob['ICD_CODE'].isin(only_cedars)])/len(cedars_prob))
print('Rows in Cedars Diagnosis File with codes shared with UCLA: ', 
      len(cedars_prob[cedars_prob['ICD_CODE'].isin(both_diag)]),'|',
      len(cedars_prob[cedars_prob['ICD_CODE'].isin(both_diag)])/len(cedars_prob))

print('Rows in UCLA Diagnosis File: ', len(ucla_prob))
print('Rows in UCLA Diagnosis File with codes unique from Cedars: ', 
      len(ucla_prob[ucla_prob['ICD_CODE'].isin(only_ucla)]),'|',
      len(ucla_prob[ucla_prob['ICD_CODE'].isin(only_ucla)])/len(ucla_prob))
print('Rows in UCLA Diagnosis File with codes shared with Cedars: ', 
      len(ucla_prob[ucla_prob['ICD_CODE'].isin(both_diag)]),'|',
      len(ucla_prob[ucla_prob['ICD_CODE'].isin(both_diag)])/len(ucla_prob))

# print(cedars_prob[cedars_prob['ICD_CODE'].isin(only_cedars)]['ICD_CODE'].value_counts()[:10])
# print(cedars_prob[cedars_prob['ICD_CODE'].isin(both_diag)]['ICD_CODE'].value_counts()[:10])

# print(ucla_prob[ucla_prob['ICD_CODE'].isin(only_ucla)]['ICD_CODE'].value_counts()[:10])
# print(ucla_prob[ucla_prob['ICD_CODE'].isin(both_diag)]['ICD_CODE'].value_counts()[:10])

#### Notes
* This one just combine via outer join

### Vitals

In [ ]:
cedars_vitals = data.load.read_files_and_combine(['Flowsheet_Vitals.txt'], args.cedars_crrt_data_dir)
ucla_vitals = data.load.read_files_and_combine(['Flowsheet_Vitals.txt'], args.ucla_crrt_data_dir)

mapping = {
        "Temp": "Temperature",
        "BMI (Calculated)": "BMI",
        "R BMI": "BMI",
        "WEIGHT/SCALE": "Weight",
        "BP": "SBP/DBP",
        "BLOOD PRESSURE": "SBP/DBP",
        "Resp": "Respirations",
        "PULSE OXIMETRY": "SpO2",

        'HEIGHT_IN': 'Height', # inches is correct
        'TEMP': 'Temperature',                  
        'O2_SATURATION': 'SpO2',
        'RESP_RATE': 'Respirations',
        'HEART_RATE': 'Pulse',
        "WEIGHT_OZ": 'Weight' # oz is correct
    }

In [ ]:
## UCLA alignment
ucla_vitals = ucla_vitals.replace({"VITAL_SIGN_TYPE": mapping})
print(len(ucla_vitals))

ucla_vitals = ucla_vitals.drop_duplicates(
        subset=["IP_PATIENT_ID", "VITAL_SIGN_TYPE", "VITAL_SIGN_TAKEN_TIME"]
    )
print(len(ucla_vitals))

ucla_vitals = ucla_vitals.apply(
        lambda col: col.str.split("/") if col.name in ["VITAL_SIGN_VALUE", "VITAL_SIGN_TYPE"] else col
    ).explode(["VITAL_SIGN_VALUE", "VITAL_SIGN_TYPE"])

print(len(ucla_vitals))

ignore_vitals = ["O2 Device"]
ignore_mask = ~ucla_vitals["VITAL_SIGN_TYPE"].isin(ignore_vitals)
ucla_vitals = ucla_vitals[ignore_mask]
print(len(ucla_vitals))

ucla_vitals["VITAL_SIGN_VALUE"] = ucla_vitals["VITAL_SIGN_VALUE"].astype(float)

########## Add to load_vitals
cedars_vitals = cedars_vitals.rename({"MEAS_NAME": "VITAL_SIGN_TYPE",
                        "RECORDED_TIME": "VITAL_SIGN_TAKEN_TIME",
                        "MEAS_VALUE": "VITAL_SIGN_VALUE"}, axis=1)

cedars_vitals = cedars_vitals.replace({'VITAL_SIGN_TYPE': mapping}) 

cedars_vitals = cedars_vitals.dropna(subset=['VITAL_SIGN_VALUE'])
print(len(ucla_vitals))

cedars_vitals = cedars_vitals.apply(
        lambda col: col.str.split("/") if col.name in ["VITAL_SIGN_VALUE", "VITAL_SIGN_TYPE"] else col
    ).explode(["VITAL_SIGN_VALUE", "VITAL_SIGN_TYPE"])

cedars_vitals = cedars_vitals.drop_duplicates(
        subset=["IP_PATIENT_ID", "VITAL_SIGN_TYPE", "VITAL_SIGN_TAKEN_TIME"]
    )

cedars_vitals["VITAL_SIGN_VALUE"] = cedars_vitals["VITAL_SIGN_VALUE"].astype(float)

In [ ]:
## BMI

bmi_df = pd.DataFrame({column: {} for column in cedars_vitals.columns})

# for a patient, if they had weight and height emasured on same day, calculate BMI
weights = cedars_vitals.loc[cedars_vitals['VITAL_SIGN_TYPE'] == 'Weight'].copy()
heights = cedars_vitals.loc[cedars_vitals['VITAL_SIGN_TYPE'] == 'Height'].copy()

weights["VITAL_SIGN_TAKEN_TIME"] = pd.to_datetime(weights["VITAL_SIGN_TAKEN_TIME"])
heights["VITAL_SIGN_TAKEN_TIME"] = pd.to_datetime(heights["VITAL_SIGN_TAKEN_TIME"])

for patient in heights['IP_PATIENT_ID'].unique():
    patient_heights = heights[heights['IP_PATIENT_ID'] == patient].copy()
    patient_weights = weights[weights['IP_PATIENT_ID'] == patient].copy()

    for j, weight in patient_weights.iterrows():
        patient_heights['TIME_DIFF'] = patient_heights['VITAL_SIGN_TAKEN_TIME'] - weight['VITAL_SIGN_TAKEN_TIME']

        selected_height = patient_heights[patient_heights['TIME_DIFF'] == patient_heights['TIME_DIFF'].min()]
        bmi = 703/16*weight['VITAL_SIGN_VALUE']/selected_height['VITAL_SIGN_VALUE']**2

        new = { 'IP_PATIENT_ID': weight['IP_PATIENT_ID'],
               'INPATIENT_DATA_ID':weight['INPATIENT_DATA_ID'],
                    'VITAL_SIGN_TAKEN_TIME': weight['VITAL_SIGN_TAKEN_TIME'],
                    'VITAL_SIGN_TYPE': 'BMI',
                    'VITAL_SIGN_VALUE' : bmi
        }
        new = pd.DataFrame(new)

        bmi_df = pd.concat([bmi_df, new])

# bmi = weight.merge(height, on=['IP_PATIENT_ID', 'VITAL_SIGN_TAKEN_TIME'], how='inner')
# bmi['BMI'] = 703/16*bmi['VITAL_SIGN_VALUE_x']/bmi['VITAL_SIGN_VALUE_y']**2
# bmi = bmi.drop(['INPATIENT_DATA_ID_y','VITAL_SIGN_TYPE_y','VITAL_SIGN_TYPE_x','VITAL_SIGN_VALUE_y','VITAL_SIGN_VALUE_x'], axis=1)
# bmi = bmi.rename(columns={'INPATIENT_DATA_ID_x':'INPATIENT_DATA_ID' ,'VITAL_SIGN_TAKEN_TIME': 'VITAL_SIGN_TAKEN_TIME', 'BMI': 'VITAL_SIGN_VALUE'})
# bmi['VITAL_SIGN_TYPE'] = 'BMI'

In [ ]:
"""
# Merge weight and height only for measurements that occured for the same patients at the same time
# Since weight and height are located the same column (VITAL_SIGN_VALUE), we must remember weight as VITAL_SIGN_VALUE_x and height as VITAL_SIGN_VALUE_y
bmi = weight.merge(
    height, on=["IP_PATIENT_ID", "VITAL_SIGN_TAKEN_TIME"], how="inner"
)

# Calculate BMI as 703*weight_in_lb/height_in_inch^2
bmi["BMI"] = 703 / 16 * bmi["VITAL_SIGN_VALUE_x"] / bmi["VITAL_SIGN_VALUE_y"] ** 2

# Drop unecessary columns due to the merge. Can probably be done in a cleaner way
bmi = bmi.drop(
    [
        "INPATIENT_DATA_ID_y",
        "VITAL_SIGN_TYPE_y",
        "VITAL_SIGN_TYPE_x",
        "VITAL_SIGN_VALUE_y",
        "VITAL_SIGN_VALUE_x",
    ],
    axis=1,
)

# Rename to align with the original dataframe
bmi = bmi.rename(
    columns={"INPATIENT_DATA_ID_x": "INPATIENT_DATA_ID", "BMI": "VITAL_SIGN_VALUE"}
)

# Fill out a the VITAL_SIGN_TYPE column with 'BMI' since all values are BMI
bmi["VITAL_SIGN_TYPE"] = "BMI"
    """

In [ ]:
cedars_vitals = pd.concat([cedars_vitals, bmi_df])
assess_value_alignment(cedars_vitals, ucla_vitals, cols=['VITAL_SIGN_TYPE'])

In [ ]:
cedars_vitals

In [ ]:
from scipy import stats
from sklearn.neighbors import KernelDensity
import numpy as np

number = []
frequency = []
ks = []

for i, patient in enumerate(sorted(cedars_vitals['IP_PATIENT_ID'].unique())):
    
    patient_subset = cedars_vitals[cedars_vitals['IP_PATIENT_ID']==patient]
    number.append(len(patient_subset))
    times = sorted(pd.DatetimeIndex(patient_subset['VITAL_SIGN_TAKEN_TIME']))
    times = sorted(list(set(times)))
    
    if len(times) < 2:
        frequency.append(0)
        ks.append(0)
        continue
    
    total_distribution = []
    for x in times:
        total_distribution.append((x-times[0]).total_seconds()/3600)

    differences = []
    for x, y in zip(times[0::], times[1::]):
        differences.append((y-x).total_seconds()/(3600))

    frequency.append(np.mean(differences))
    ks.append(stats.kurtosis(total_distribution))
    
    if i < 5:
        print(differences)
        print(total_distribution)
        print(ks[-1])
        all_freq = np.array(total_distribution)[:,None]
        X_plot = np.linspace(-5, 20, 1000)[:, np.newaxis]
        kde = KernelDensity(kernel='gaussian', bandwidth=1.0).fit(all_freq)
        log_dens = kde.score_samples(X_plot)
        plt.plot(
            X_plot[:, 0],
            np.exp(log_dens)
        )
        plt.show()



In [ ]:
freq = pd.DataFrame({'IP_PATIENT_ID': sorted(cedars_vitals['IP_PATIENT_ID'].unique()),
                     'VISITS': number,
                     'FREQ': frequency,
                     'KS':ks,
                     })

display(freq['VISITS'].describe())
display(freq['FREQ'].describe())
display(freq['KS'].describe())


In [ ]:
from scipy import stats
from sklearn.neighbors import KernelDensity
import numpy as np

number = []
frequency = []
ks = []

for i, patient in enumerate(sorted(ucla_vitals['IP_PATIENT_ID'].unique())):
    
    patient_subset = ucla_vitals[ucla_vitals['IP_PATIENT_ID']==patient]
    number.append(len(patient_subset))
    times = sorted(pd.DatetimeIndex(patient_subset['VITAL_SIGN_TAKEN_TIME']))
    times = sorted(list(set(times)))
    
    if len(times) < 2:
        frequency.append(0)
        ks.append(0)
        continue
    
    total_distribution = []
    for x in times:
        total_distribution.append((x-times[0]).total_seconds()/3600)

    differences = []
    for x, y in zip(times[0::], times[1::]):
        differences.append((y-x).total_seconds()/(3600))

    frequency.append(np.mean(differences))
    ks.append(stats.kurtosis(total_distribution))
    
    if i < 5:
        print(differences)
        print(total_distribution)
        print(ks[-1])
        all_freq = np.array(total_distribution)[:,None]
        X_plot = np.linspace(-5, 20, 1000)[:, np.newaxis]
        kde = KernelDensity(kernel='gaussian', bandwidth=1.0).fit(all_freq)
        log_dens = kde.score_samples(X_plot)
        plt.plot(
            X_plot[:, 0],
            np.exp(log_dens)
        )
        plt.show()



In [ ]:
freq = pd.DataFrame({'IP_PATIENT_ID': sorted(ucla_vitals['IP_PATIENT_ID'].unique()),
                     'VISITS': number,
                     'FREQ': frequency,
                     'KS':ks,
                     })

display(freq['VISITS'].describe())
display(freq['FREQ'].describe())
display(freq['KS'].describe())

#### Notes
* UCLA has an explicit measure for BMI. Cedars does not have this, but it can be calculated from height and weight. The rule is that when there are measurements for height and weight at the EXACT same time, calculate the BMI. I found that we should be strict about time, because there are cases where weight can be measured multiple times a day with changes
* UCLA is more balanced between measures
* Cedars has fewer tempearture, weight, height, and thus BMI measures
* Of the 482 height measures, 424 have corresponding weight measures at the same time


*  whenever we have height, we can characterize BMI
* check average frequency of patient/ fixed frequency?

In [ ]:
plt.figure(figsize=[15,15])

plt.subplot(1,2,1)
pie_plot(cedars_vitals['VITAL_SIGN_TYPE'].value_counts().reindex(ucla_vitals['VITAL_SIGN_TYPE'].value_counts().index), title="Cedars")
print(cedars_vitals['VITAL_SIGN_TYPE'].value_counts(normalize=True))
plt.subplot(1,2,2)
pie_plot(ucla_vitals['VITAL_SIGN_TYPE'].value_counts(), title="UCLA")


In [ ]:
print(len(cedars_vitals))
print(len(cedars_vitals['IP_PATIENT_ID'].unique()))
print(len(ucla_vitals))
print(len(ucla_vitals['IP_PATIENT_ID'].unique()))

### Procedures

In [ ]:
cedars_proc = data.load.read_files_and_combine(['Procedures.txt'], args.cedars_crrt_data_dir)
ucla_proc = data.load.read_files_and_combine(['Procedures.txt'], args.ucla_crrt_data_dir)
proc_mapping = pd.read_excel(os.path.join(args.cedars_crrt_data_dir, 'karumanchi_00001867_proc_code_bridge_table.xlsx'))

# Cedars alignment. Assume contact date is diagnosis date since that's the date we have
cedars_proc = cedars_proc.rename(
        {
        # Control
        "PROCEDURE_CODE": "PROC_CODE", 
        "PROCEDURE_DATE": "PROC_DATE",

        # Cedars 
        "PROC_START_TIME": "PROC_DATE"}, axis=1
    )

cedars_proc['PROC_CODE'] = cedars_proc['PROC_CODE'].astype(str) # Already CPT: {99195, 93925, 92950}
ucla_proc['PROC_CODE'] = ucla_proc['PROC_CODE'].astype(str)
proc_mapping['PROC_CODE'] = proc_mapping['PROC_CODE'].astype(str)
proc_mapping['OT_PROC_CODE'] = proc_mapping['OT_PROC_CODE'].astype(str)

assess_value_alignment(cedars_proc, ucla_proc, cols=['PROC_CODE'], len_only=True)

##### Naive Mapping
* Naive mapping just uses PROC_CODE as the key and OT_PROC_CODE as the value
* Note that OT_CODE_TYPE can have different values: CPT, NaN, CUSTOM, HCPCS, ASA, ADA, HIPPS
* We naively consider everything a CPT code if it is not CUSTOM

In [ ]:
display(proc_mapping)

In [ ]:
cpt_mapping = proc_mapping
unique_codes = set(cedars_proc['PROC_CODE'].unique()).difference(set(cpt_mapping['PROC_CODE'].unique()))
intersection = set(cedars_proc['PROC_CODE'].unique()).intersection(set(cpt_mapping['PROC_CODE'].unique()))

print('Number of Mappings: ', len(proc_mapping))
print('Number of Unique Cedars Codes that exist in mapping file', len(intersection))
print('Number of Unique Cedars Codes that do not exist in mapping file: ', len(unique_codes))

cpt_mapping = proc_mapping[~proc_mapping['OT_CODE_TYPE'].isin(['CUSTOM'])]
unique_codes = set(cedars_proc['PROC_CODE'].unique()).difference(set(cpt_mapping['PROC_CODE'].unique()))
intersection = set(cedars_proc['PROC_CODE'].unique()).intersection(set(cpt_mapping['PROC_CODE'].unique()))
cpt_mapping = cpt_mapping[cpt_mapping['PROC_CODE'].isin(cedars_proc['PROC_CODE'].unique())]

print('Number of Unique Cedars Codes with direct CPT Mapping: ', len(intersection))
print('Number of Unique Cedars Codes without direct CPT Mapping: ', len(unique_codes))

mapping_dict_naive = dict(
    zip(cpt_mapping['PROC_CODE'], cpt_mapping['OT_PROC_CODE'])
)

cedars_proc["PROC_CODE"] = cedars_proc["PROC_CODE"].replace(
    mapping_dict_naive
    )

assess_value_alignment(cedars_proc, ucla_proc, cols=['PROC_CODE'], len_only=True)

##### Notes 
* We see that there are still some custom cedars codes with no mapping to CPT codes
* Also, it seems that (see below), there are cases when a Cedars code maps to both a Cedars code AND a CPT code. 
* The bad case below shows a case when there is no CPT code, but there are some cases where there are CPT codes but maybe they are also 'CUSTOM' type, and the mapping fails because the first mapping automatically obtained by pandas is the wrong code
* To fix this below, we individually find ALL the mappings for each Cedars procedure code, and then check the OT_PROC_CODE if it is a CPT code (regardless of OT_CODE_TYPE)

In [ ]:
display(proc_mapping[proc_mapping['PROC_CODE'] == 'IMG117'])# GOOD example
display(proc_mapping[proc_mapping['PROC_CODE'] == 'IMG1534']) # BAD example

##### Non Naive
* We individually find ALL the mappings for each Cedars procedure code, and then check the OT_PROC_CODE if it is a CPT code (regardless of OT_CODE_TYPE)

In [ ]:
from hcuppy.cpt import CPT
from pandas import DataFrame
from typing import List


def trace_code_connections(starting_code: str, mapping_df: DataFrame) -> List[str]:
    """
    Find all the different codes that a starting code is related to, as documented in the mapping_df
    Each PROC_CODE is associated with an OT_PROC_CODE, but mapping is not one-to-one nor one-way
    Perform a depth first search to find all codes that a given code is 'connected' to
    """

    cpt = CPT()

    found_code = "na"

    visited_codes = set()
    codes_to_visit = {starting_code}

    while len(codes_to_visit) > 0:

        current_code = codes_to_visit.pop()

        if current_code not in visited_codes:

            cpt_code = cpt.get_cpt_section(current_code)

            # If there is one cpt code, store it
            if cpt_code["sect"] != "na":
                found_code = current_code
                break

            # get related codes from OT_PROC_CODE when the current code is PROC_CODE
            # AND get related codes from PROC_CODE when the current code is OT_PROC_CODE
            neighbours_forwards = mapping_df[mapping_df["PROC_CODE"] == current_code][
                "OT_PROC_CODE"
            ].to_list()
            neighbours_backwards = mapping_df[
                mapping_df["OT_PROC_CODE"] == current_code
            ]["PROC_CODE"].to_list()

            neighbours_forwards = [x for x in neighbours_forwards if str(x) != "nan"]
            neighbours_backwards = [x for x in neighbours_forwards if str(x) != "nan"]
            additions = set(neighbours_backwards).union(set(neighbours_forwards))

            codes_to_visit = set(codes_to_visit).union(additions)

            visited_codes.add(current_code)

    return found_code

original_codes = []
cpt_codes = []

for i, code in enumerate(cedars_proc['PROC_CODE'].unique()):

    # Trace all codes
    found_code = trace_code_connections(code, proc_mapping)
    
    original_codes.append(code)
    cpt_codes.append(found_code)

mapping_dict = dict(
    zip(original_codes, cpt_codes)
)

cedars_proc["PROC_CODE"] = cedars_proc["PROC_CODE"].replace(
    mapping_dict
    )

assess_value_alignment(cedars_proc, ucla_proc, cols=['PROC_CODE'], len_only=True)

##### Notes
* The number of unique Cedars codes drops, showing many more now have mappings to CPT codes.
* But the number of intersecting codes does not increase. Which means that many of the Cedars codes map to the same CPT codes 

In [ ]:
from hcuppy.cpt import CPT
from pickle import load
from os.path import isfile
from pandas import DataFrame

def map_proc_code_to_cpt(
    static_df: DataFrame,
    raw_data_dir: str,
    proc_mapping_file: str = "Procedures_Code_Mapping.pkl",
) -> DataFrame:

    # Should only do for Cedars
    if not isfile(join(raw_data_dir, proc_mapping_file)):
        return static_df

    with open(join(raw_data_dir, proc_mapping_file), "rb") as f:
        loaded_dict = load(f)

    static_df["PROC_CODE"] = static_df["PROC_CODE"].astype(str)

    static_df["PROC_CODE"] = static_df["PROC_CODE"].replace(loaded_dict)

    return static_df

cedars_proc = map_proc_code_to_cpt(cedars_proc, args.cedars_crrt_data_dir)

cpt = CPT()

cedars_proc_cpt = data.longitudinal_utils.hcuppy_map_code(
        cedars_proc,
        code_col="PROC_CODE",
        exploded_cols=["CPT_SECTION", "SECTION_DESCRIPTION"],
        hcuppy_converter_function=cpt.get_cpt_section,
    )

ucla_proc_cpt = data.longitudinal_utils.hcuppy_map_code(
        ucla_proc,
        code_col="PROC_CODE",
        exploded_cols=["CPT_SECTION", "SECTION_DESCRIPTION"],
        hcuppy_converter_function=cpt.get_cpt_section,
    )


In [ ]:
cedars_proc_code = set(cedars_proc['PROC_CODE'])
ucla_proc_code = set(ucla_proc['PROC_CODE'])

only_cedars = cedars_proc_code.difference(ucla_proc_code)
only_ucla = ucla_proc_code.difference(cedars_proc_code)
both_diag = cedars_proc_code.intersection(ucla_proc_code)

print('Rows in Cedars Diagnosis File: ', len(cedars_proc))
print('Rows in Cedars Diagnosis File with codes unique from UCLA: ', 
      len(cedars_proc[cedars_proc['PROC_CODE'].isin(only_cedars)]), '|',
      len(cedars_proc[cedars_proc['PROC_CODE'].isin(only_cedars)])/len(cedars_proc))
print('Rows in Cedars Diagnosis File with codes shared with UCLA: ', 
      len(cedars_proc[cedars_proc['PROC_CODE'].isin(both_diag)]),'|',
      len(cedars_proc[cedars_proc['PROC_CODE'].isin(both_diag)])/len(cedars_proc))

print('Rows in UCLA Diagnosis File: ', len(ucla_proc))
print('Rows in UCLA Diagnosis File with codes unique from Cedars: ', 
      len(ucla_proc[ucla_proc['PROC_CODE'].isin(only_ucla)]),'|',
      len(ucla_proc[ucla_proc['PROC_CODE'].isin(only_ucla)])/len(ucla_proc))
print('Rows in UCLA Diagnosis File with codes shared with Cedars: ', 
      len(ucla_proc[ucla_proc['PROC_CODE'].isin(both_diag)]),'|',
      len(ucla_proc[ucla_proc['PROC_CODE'].isin(both_diag)])/len(ucla_proc))


In [ ]:
print("Number of Cedars Rows: ", len(cedars_proc_cpt))
print("Number of Cedars Unique CPT: ", len(cedars_proc_cpt["CPT_SECTION"].unique()))
print(cedars_proc_cpt["CPT_SECTION"].value_counts())

print("Number of UCLA Rows: ", len(ucla_proc_cpt))
print("Number of UCLA Unique CPT: ",len(ucla_proc_cpt["CPT_SECTION"].unique()))
print(ucla_proc_cpt["CPT_SECTION"].value_counts())


##### Notes
* Of the ~1 million rows, about 700 thousand are mapped to actual cpt codes
* The variety of CPT codes from cedars is much lower

### Medication

In [ ]:
cedars_meds = data.load.read_files_and_combine(['Medications.txt'], args.cedars_crrt_data_dir)
ucla_meds = data.load.read_files_and_combine(['Medications.txt'], args.ucla_crrt_data_dir)

cedars_meds = cedars_meds.dropna(subset=["PHARM_SUBCLASS"])
ucla_meds = ucla_meds.dropna(subset=["PHARM_SUBCLASS"])

for column in  ['PHARM_SUBCLASS']:
    ucla_meds[column] = ucla_meds[column].str.upper()
for column in  ['PHARM_SUBCLASS']:
    cedars_meds[column] = cedars_meds[column].str.upper()

cedars_meds = cedars_meds.rename({
                          "ORDERING_DATE": "ORDER_DATE",
                          "NAME": 'MEDICATION_NAME'}, axis=1)

ucla_meds = ucla_meds.rename({"MEDISPAN_CLASS_NAME": "PHARM_CLASS",
                          "ORDERING_DATE": "ORDER_DATE"}, axis=1)

ucla_meds = ucla_meds[~ucla_meds['PHARM_SUBCLASS'].str.contains('EACH')] #277
ucla_meds = ucla_meds[~ucla_meds['PHARM_SUBCLASS'].str.isnumeric()] # 251

assess_value_alignment(cedars_meds, ucla_meds, cols=['PHARM_SUBCLASS'], len_only=True)

In [ ]:
left = (set(cedars_meds['PHARM_SUBCLASS'].unique())).difference(set(ucla_meds['PHARM_SUBCLASS'].unique()))
right = set(ucla_meds['PHARM_SUBCLASS'].unique())
distsort = fuzzy_pairwise_diff(left, right, mode='sort')
distpart = fuzzy_pairwise_diff(left, right, mode='partial')
dist = fuzzy_pairwise_diff(left, right)
print(distsort)
print(distpart)
print(dist)

##### Notes
* 'sorted' distance looks for strings that have the same words but shifted. If these are very similar, then we consider them the same
* 'partial' distance looks for strings that are subsets of another. These need review
* Then we consider singular character differences. These also need review, but can be the same (e.g., singular vs plural) BUT also very different (e.g., Indirect vs Direct)

* What I do below is that I look at the three different string matchings, and if similarity is over 0.9, i consider it a good mapping. I save the output, and then manually inspect for required changes. For the medications without mapping, I manually looked for mappings by considering both the top choices from the similarity metrics above (maybe they just barely were below the threshold) or I manually looked through the UCLA medications

* From the 245 unique cedars medications, about 100 are automatically mapped, about 100 needed manual mapping, and about 40 do not have mappings

In [ ]:
import difflib
import pickle 

def create_mapping_dict(left, right):

    exceptions = {
        "ANTIDEPRESSANT - SEROTONIN-2 ANTAGONIST-REUPTAKE INHIBITORS (SARIS)",
        "ANTIDEPRESSANT-NOREPINEPHRINE AND DOPAMINE REUPTAKE INHIBITORS (NDRIS)",
        "BLOOD CELL AND PLATELET DISORDER TX-SPLEEN TYROSINE KINASE INHIBITORS",
        "HYPERURICEMIA THERAPY - XANTHINE OXIDASE INHIBITORS",
        "NARCOLEPSY THERAPY AGENTS - NON-SYMPATHOMIMETIC",
        "URINARY RETENTION THERAPY - PARASYMPATHOMIMETIC AGENTS",
        "CALCIMIMETIC, PARATHYROID CALCIUM RECEPTOR SENSITIVITY ENHANCER",
    }

    manual_mapping = {
        "CALCIUM CHANNEL BLOCKERS - DIHYDROPYRIDINES - CEREBROVASCULAR SPECIFIC": "CALCIUM CHANNEL BLOCKERS",
        "CALCIUM CHANNEL BLOCKERS - BENZOTHIAZEPINES": "CALCIUM CHANNEL BLOCKERS",
        "BETA BLOCKERS NON-CARDIAC SELECTIVE": "BETA BLOCKERS NON-SELECTIVE",
        "ASTHMA THERAPY - LEUKOTRIENE RECEPTOR ANTAGONISTS": "LEUKOTRIENE MODULATORS",
        "CMV ANTIVIRAL AGENT - INORGANIC PYROPHOSPHATE ANALOGS": "CMV AGENTS",
        "OVERACTIVE BLADDER AGENTS - BETA -3 ADRENERGIC RECEPTOR AGONIST": "URINARY ANTISPASMODICS - BETA-3 ADRENERGIC AGONISTS 61",
        "ANTIPARKINSON - DOPAMINERG-PERIPHERAL DOPA-DECARBOXYLASE INHIBIT COMB": "ANTIPARKINSON DOPAMINERGICS",
        "CONTRAST MEDIA - BARIUM": "RADIOGRAPHIC CONTRAST MEDIA",
        "ANTIPSYCHOTIC - ATYPICAL DOPAMINE-SEROTONIN ANTAG- BENZISOTHIAZOLONES": "BENZODIAZEPINE ANTAGONISTS",
        "THROMBOLYTIC - TISSUE PLASMINOGEN ACTIVATORS": "THROMBOLYTIC ENZYMES",
        "ANTICONVULSANT - BARBITURATES AND DERIVATIVES": "ANTICONVULSANTS - MISC.",
        "ANTIHISTAMINES - 2ND GENERATION": "ANTIHISTAMINES - NON-SEDATING",
        "CONTRAST MEDIA - MAGNETIC RESONANCE GADOLINIUM COMPLEXES": "RADIOGRAPHIC CONTRAST MEDIA",
        "ANTIDEPRESSANT-TRICYCLICS AND RELATED (NON-SELECT REUPTAKE INHIBITORS)": "TRICYCLIC AGENTS",
        "ANTIARRHYTHMIC - CLASS IV": "ANTIARRHYTHMICS - MISC.",
        "AGENTS FOR OPIOID WITHDRAWAL, OPIOID-TYPE": "OPIOID PARTIAL AGONISTS",
        "ANTIANXIETY AGENT - ANTIHISTAMINE TYPE": "ANTIANXIETY AGENTS - MISC.",
        "ANTISEPTIC - OXIDIZING AGENTS": "ANTISEPTICS & DISINFECTANTS",
        "VACCINE VIRAL - INFLUENZA A AND B": "VIRAL VACCINES",
        "CONTRAST MEDIA - IODINATED NONIONIC": "RADIOGRAPHIC CONTRAST MEDIA",
        "DERMATOLOGICAL - PROTECTANTS": "MISC. DERMATOLOGICAL PRODUCTS",
        "DERMATOLOGICAL - ANTIBACTERIAL MIXTURES": "MISC. DERMATOLOGICAL PRODUCTS",
        "ANTIDOTE - CYANIDE POISONING": "ANTIDOTES - CHELATING AGENTS",
        "PENICILLIN ANTIBIOTIC - NATURAL": "NATURAL PENICILLINS",
        "ANTISEPTIC - CHLORINE RELEASING": "CHLORINE ANTISEPTICS",
        "DEXTROSE SOLUTIONS": "PERITONEAL DIALYSIS SOLUTIONS",
        "DERMATOLOGICAL - ENZYME COMBINATIONS OTHER": "ENZYMES",
        "ANTIANXIETY AGENT - NON-BENZODIAZEPINE": "ANTIANXIETY AGENTS - MISC.",
        "ANTICONVULSANT - CARBOXYLIC ACID DERIVATIVES": "ANTICONVULSANTS - MISC.",
        "ANTIARRHYTHMIC - CLASS IB": "ANTIARRHYTHMICS - MISC.",
        "CEPHALOSPORIN ANTIBIOTICS - 1ST GENERATION": "CEPHALOSPORINS - 1ST GENERATION",
        "LAXATIVE - BULK FORMING": "BULK LAXATIVES",
        "ASTHMA THERAPY - INHALED CORTICOSTEROIDS (GLUCOCORTICOIDS)": "GLUCOCORTICOSTEROIDS",
        "ASTHMA/COPD THERAPY - BETA ADRENERGIC-GLUCOCORTICOID COMBINATIONS": "GLUCOCORTICOSTEROIDS",
        "DERMATOLOGICAL - GLUCOCORTICOID": "GLUCOCORTICOSTEROIDS",
        "ANTINEOPLASTIC - CD20 SPECIFIC RECOMBINANT MONOCLONAL ANTIBODY AGENTS": "MONOCLONAL ANTIBODIES",
        "ANTINEOPLASTIC - PHOTOSENSITIZERS": "ANTINEOPLASTICS MISC.",
        "ACNE THERAPY TOPICAL - ANTI-INFECTIVE": "ANTI-INFECTIVE AGENTS - MISC.",
        "ANALGESIC OPIOID HYDROMORPHONE COMBINATIONS": "ANALGESIC COMBINATIONS",
        "CONTRAST MEDIA - IODINATED IONIC": "RADIOGRAPHIC CONTRAST MEDIA",
        "ANTICONVULSANT - FUNCTIONALIZED AMINO ACID": "ANTICONVULSANTS - MISC.",
        "ANTIARRHYTHMIC - CLASS II": "ANTIARRHYTHMICS - MISC.",
        "MINERALS AND ELECTROLYTES - IODINE": "ELECTROLYTE MIXTURES",
        "VITAMINS - B-1, THIAMINE AND DERIVATIVES": "B-COMPLEX VITAMINS",
        "VITAMINS - B-6, PYRIDOXINE AND DERIVATIVES": "B-COMPLEX VITAMINS",
        "VITAMINS - FOLIC ACID AND DERIVATIVES": "B-COMPLEX VITAMINS",
        "ALTERNATIVE THERAPY - UNCLASSIFIED": "ALTERNATIVE MEDICINE - U",
        "ANTIPSYCHOTIC - ATYPICAL DOPAMINE-SEROTONIN ANTAG-DIBENZODIAZEPINE DER": "ANTIPSYCHOTICS - MISC.",
        "IMMUNOSUPPRESSIVE - CALCINEURIN INHIBITORS": "IMMUNOSUPPRESSIVE AGENTS",
        "ANTIPARKINSON THERAPY - NON-ERGOT DOPAMINE AGONIST AGENTS": "ANTIPARKINSON DOPAMINERGICS",
        "HEPATITIS C - NUCLEOSIDE ANALOGS": "HEPATITIS AGENTS",
        "HEPATITIS B TREATMENT- NUCLEOSIDE ANALOGS (ANTIVIRAL)": "HEPATITIS AGENTS",
        "ANALGESIC OPIOID CODEINE COMBINATIONS": "ANALGESIC COMBINATIONS",
        "ANORECTAL - GLUCOCORTICOIDS": "GLUCOCORTICOSTEROIDS",
        "IMMUNOSUPPRESSIVE - PURINE ANALOGS": "IMMUNOSUPPRESSIVE AGENTS",
        "OPHTHALMIC - ANTI-INFLAMMATORY, NSAIDS": "NONSTEROIDAL ANTI-INFLAMMATORY AGENTS (NSAIDS)",
        "RINGER'S AND LACTATED RINGER'S SOLUTIONS": "IRRIGATION SOLUTIONS",
        "DEXTROSE AND LACTATED RINGER'S SOLUTIONS": "IRRIGATION SOLUTIONS",
        "NSAID ANALGESIC, CYCLOOXYGENASE-2 (COX-2) SELECTIVE INHIBITORS": "NONSTEROIDAL ANTI-INFLAMMATORY AGENTS (NSAIDS)",
        "CEPHALOSPORIN ANTIBIOTICS - 4TH GENERATION": "CEPHALOSPORINS - 4TH GENERATION",
        "HEMOSTATIC TOPICAL AGENTS": "HEMOSTATICS - TOPICAL",
        "OPHTHALMIC - ANTICHOLINERGICS": "OPHTHALMICS - MISC.",
        "SEDATIVE-HYPNOTIC - GABA-RECEPTOR MODULATORS": "HORMONE RECEPTOR MODULATORS",
        "MACROLIDE ANTIBIOTICS": "ANTIBIOTICS - TOPICAL",
        "ANTIBACTERIAL FOLATE ANTAGONIST - OTHER COMBINATIONS": "ANTI-INFECTIVE MISC. - COMBINATIONS",
        "CEPHALOSPORIN ANTIBIOTICS - 2ND GENERATION": "CEPHALOSPORINS - 2ND GENERATION",
        "ANTIPSYCHOTIC -ATYPICAL DOPAMINE-SEROTONIN ANTAG-DIBENZOTHIAZEPINE DER": "ANTIPSYCHOTICS - MISC.",
        "CEPHALOSPORIN ANTIBIOTICS - 3RD GENERATION": "CEPHALOSPORINS - 3RD GENERATION",
        "GALLSTONE SOLUBILIZING (LITHOLYSIS) AGENTS": "GALLSTONE SOLUBILIZING AGENTS",
        "AGENTS TO TREAT HYPOGLYCEMIA (HYPERGLYCEMICS)": "ANTIHYPERLIPIDEMICS - COMBINATIONS",
        "DERMATOLOGICAL - ANTIBACTERIAL POVIDONE-IODINE PREPARATIONS": "MISC. DERMATOLOGICAL PRODUCTS",
        "ANGIOTENSIN II RECEPTOR BLOCKERS (ARBS)": "ANGIOTENSIN II RECEPTOR ANTAGONISTS",
        "ATTENTION DEFICIT-HYPERACTIVITY (ADHD) THERAPY, STIMULANT-TYPE": "ATTENTION-DEFICIT/HYPERACTIVITY DISORDER (ADHD) AGENTS",
        "ANTICONVULSANT - MONOSACCHARIDE DERIVATIVES": "ANTICONVULSANTS - MISC.",
        "ANTICONVULSANT - IMINOSTILBENE DERIVATIVES": "ANTICONVULSANTS - MISC.",
        "HERPES ANTIVIRAL AGENT - PURINE ANALOGS": "HERPES AGENTS",
        "ANTIHYPERGLYCEMIC, INCRETIN MIMETIC,GLP-1 RECEPTOR AGONIST ANALOG-TYPE": "INCRETIN MIMETIC AGENTS (GLP-1 RECEPTOR AGONISTS)",
        "PULMONARY FIBROSIS TREATMENT AGENTS - ANTIFIBROTIC THERAPY": "PULMONARY FIBROSIS AGENTS",
        "CMV ANTIVIRAL AGENT - NUCLEOSIDE ANALOGS": "CMV AGENTS",
        "GLUCOCORTICOIDS": "GLUCOCORTICOSTEROIDS",
        "ANTIARRHYTHMIC - CLASS III": "ANTIARRHYTHMICS TYPE III",
        "MU-OPIOID RECEPTOR ANTAGONISTS, PERIPHERALLY-ACTING": "PERIPHERAL OPIOID RECEPTOR ANTAGONISTS",
        "GENERAL ANESTHETIC - PARENTERAL, PHENOL DERIVATIVES": "ANESTHETICS - MISC.",
        "MINERALS AND ELECTROLYTES - PARENTERAL ELECTROLYTE COMBINATIONS": "ELECTROLYTE MIXTURES",
        "LOW MOLECULAR WEIGHT HEPARINS": "HEPARINS AND HEPARINOID-LIKE AGENTS",
        "ANTIEMETIC - CANNABINOID TYPE": "ANTIEMETICS - MISCELLANEOUS",
        "ANTIEMETIC - SELECTIVE SEROTONIN 5-HT3 ANTAGONISTS": "5-HT3 RECEPTOR ANTAGONISTS",
        "MINERALS AND ELECTROLYTES - PARENTERAL ELECTROLYTE COMBINATIONS": "ELECTROLYTE MIXTURES",
        "OXYTOCIC - OXYTOCIN AND ANALOGS": "OXYTOCICS",
        "NSAID ANALGESICS (COX NON-SPECIFIC) - PROPIONIC ACID DERIVATIVES": "NONSTEROIDAL ANTI-INFLAMMATORY AGENTS (NSAIDS)",
        "DERMATOLOGICAL - PROTECTANT COMBINATIONS": "MISC. DERMATOLOGICAL PRODUCTS",
        "HEMOSTATIC SYSTEMIC - ANTIFIBRINOLYTIC AGENTS": "HEMOSTATICS - SYSTEMIC",
        "PULMONARY ANTIHYPERTENSIVE AGENTS-SOLUBLE GUANYLATE CYCLASE STIMULATOR": "PULMONARY HYPERTENSION - SOL GUANYLATE CYCLASE STIMULATOR",
        "BIPOLAR THERAPY AGENTS - ATYPICAL ANTIPSYCHOTICS": "ANTIPSYCHOTICS - MISC.",
        "ANALGESIC OR ANTIPYRETIC NON-OPIOID": "ANALGESICS OTHER",
        "ANTIHISTAMINE - 1ST GENERATION - PIPERIDINES": "ANTIHISTAMINES - PIPERIDINES",
        "LAXATIVE - SALINE AND OSMOTIC": "SALINE LAXATIVES",
        "ANTACID - SIMETHICONE COMBINATIONS": "ANTACID COMBINATIONS",
        "ANALGESIC OPIOID OXYCODONE COMBINATIONS": "ANALGESIC COMBINATIONS",
        "DIURETIC - THIAZIDES AND RELATED": "THIAZIDES AND THIAZIDE-LIKE DIURETICS",
        "NASAL CORTICOSTEROIDS": "NASAL STEROIDS",
        "DERMATOLOGICAL - CALCINEURIN INHIBITORS": "MISC. DERMATOLOGICAL PRODUCTS",
        "HYPERPARATHYROID TREATMENT AGENTS - VITAMIN D ANALOG-TYPE": "ANTITHYROID AGENTS",
        "OPHTHALMIC - DIAGNOSTIC AGENTS": "OPHTHALMICS - MISC.",
        "PLATELET AGGREGATION INHIB-PDESTERASE AND ADENOSINE DEAMINASE INHIBITR": "PLATELET AGGREGATION INHIBITORS",
        "ANTIHISTAMINE - 1ST GENERATION - ETHANOLAMINES": "ANTIHISTAMINES - ETHANOLAMINES",
        "ANTIDEPRESSANT - ALPHA-2 RECEPTOR ANTAGONISTS (NASSA)": "ALPHA-2 RECEPTOR ANTAGONISTS (TETRACYCLICS)",
        "PHARMACEUTICAL ADJUVANT - FLAVORING AGENTS": "PHARMACEUTICAL EXCIPIENTS",
        "ANTICONVULSANT - PYRROLIDINE DERIVATIVES": "ANTICONVULSANTS - MISC.",
        "ANTIPROTOZOAL-ANTIBACTERIAL 1ST GENERATION 2-METHYL-5-NITROIMIDAZOLE": "ANTIPROTOZOAL AGENTS",
        "PULMONARY ARTERIAL HYPERTENSION - SELECTIVE CGMP-PDE5 INHIBITORS": "PULMONARY HYPERTENSION - PHOSPHODIESTERASE INHIBITORS",
        "CMV ANTIVIRAL AGENT - TERMINASE COMPLEX INHIBITORS": "CMV AGENTS",
    }

    # Got these from running the algorithm. Just saving here to save time for future reference
    mapping = {
        "CALCIUM CHANNEL BLOCKERS - DIHYDROPYRIDINES": "CALCIUM CHANNEL BLOCKERS",
        "BETA BLOCKERS CARDIAC SELECTIVE": "BETA BLOCKERS CARDIO-SELECTIVE",
        "INFLAMMATORY BOWEL AGENT - AMINOSALICYLATES AND RELATED AGENTS": "SALICYLATES",
        "DERMATOLOGICAL - TOPICAL LOCAL ANESTHETIC AMIDES": "LOCAL ANESTHETICS - AMIDES",
        "DEXTROSE AND SODIUM CHLORIDE SOLUTIONS": "SODIUM",
        "MINERALS AND ELECTROLYTES - POTASSIUM FOR INJECTION": "POTASSIUM",
        "ANTIFUNGAL - FLUORINATED PYRIMIDINE-TYPE AGENTS": "ANTIFUNGALS",
        "DERMATOLOGICAL - LOCAL ANESTHETIC COMBINATIONS": "LOCAL ANESTHETIC COMBINATIONS",
        "ANTIFUNGAL - GLUCAN SYNTHESIS INHIBITOR, ECHINOCANDINS": "ANTIFUNGAL - GLUCAN SYNTHESIS INHIBITORS",
        "ANALGESIC OPIOID AGONISTS": "OPIOID AGONISTS",
        "PHOSPHATE BINDERS": "PHOSPHATE",
        "ANTIANGINAL - CORONARY VASODILATORS (NITRATES)": "NITRATES",
        "ARTIFICIAL TEARS AND LUBRICANT SINGLE AGENTS": "ARTIFICIAL TEARS AND LUBRICANTS",
        "ANTIANXIETY AGENT - BENZODIAZEPINES": "BENZODIAZEPINES",
        "DERMATOLOGICAL - BURN PRODUCTS ANTI-INFECTIVE": "BURN PRODUCTS",
        "CARDIOVASCULAR SYMPATHOMIMETICS": "SYMPATHOMIMETICS",
        "DIURETIC - LOOP": "LOOP DIURETICS",
        "MINERALS AND ELECTROLYTES - POTASSIUM, ORAL": "POTASSIUM",
        "ANTACID - CALCIUM": "CALCIUM",
        "DIURETIC - CARBONIC ANHYDRASE INHIBITORS": "CARBONIC ANHYDRASE INHIBITORS",
        "METABOLIC MODIFIER - CARNITINE REPLENISHER AGENTS": "METABOLIC MODIFIERS",
        "DIURETIC - SELECTIVE ARGININE VASOPRESSIN V2 RECEPTOR ANTAGONISTS": "VASOPRESSIN RECEPTOR ANTAGONISTS",
        "OPHTHALMIC-INTRAOCULAR PRESSURE REDUCING AGENTS, PROSTAGLANDIN ANALOGS": "PROSTAGLANDINS",
        "OPHTHALMIC ANTIBIOTIC - FLUOROQUINOLONES": "FLUOROQUINOLONES",
        "INSULIN ANALOGS - RAPID ACTING": "INSULIN",
        "MOUTH AND THROAT - ANTISEPTICS": "ANTISEPTICS - MOUTH/THROAT",
        "MINERALS AND ELECTROLYTES - CALCIUM REPLACEMENT": "CALCIUM",
        "DERMATOLOGICAL - TOPICAL LOCAL ANESTHETIC ESTERS": "LOCAL ANESTHETICS - ESTERS",
        "SMOKING DETERRENTS - NICOTINE-TYPE": "SMOKING DETERRENTS",
        "HEPARINS": "HEPARINS AND HEPARINOID-LIKE AGENTS",
        "ABORTIFACIENTS OR CERVICAL RIPENING AGENTS - PROSTAGLANDIN ANALOGS": "PROSTAGLANDINS",
        "URINARY ANTISPASMODIC - SMOOTH MUSCLE RELAXANTS": "ANTISPASMODICS",
        "ANTICOAGULANTS - COUMARIN": "COUMARIN ANTICOAGULANTS",
        "INDIRECT FACTOR XA INHIBITORS": "DIRECT FACTOR XA INHIBITORS",
        "ANTIHISTAMINE - 1ST GENERATION - PHENOTHIAZINES": "PHENOTHIAZINES",
        "ANTIPSYCHOTIC - BUTYROPHENONE DERIVATIVES": "BUTYROPHENONES",
        "INSULINS": "INSULIN",
        "ANTITUBERCULAR - ISONICOTINIC ACID DERIVATIVES": "NICOTINIC ACID DERIVATIVES",
        "ARTIFICIAL TEARS AND LUBRICANT COMBINATIONS": "ARTIFICIAL TEARS AND LUBRICANTS",
        "SYSTEMIC SYMPATHOMIMETIC DECONGESTANTS": "SYMPATHOMIMETIC DECONGESTANTS",
        "MINERALS AND ELECTROLYTES - IRON": "IRON",
        "AMINOGLYCOSIDE ANTIBIOTIC": "AMINOGLYCOSIDES",
        "DMARD - PYRIMIDINE SYNTHESIS INHIBITORS": "PYRIMIDINE SYNTHESIS INHIBITORS",
        "PLATELET AGGREGATION INHIBITORS - PHOSPHODIESTERASE III INHIBITORS": "PLATELET AGGREGATION INHIBITORS",
        "DIAGNOSTIC RADIOPHARMACEUTICALS - RENAL IMAGING": "DIAGNOSTIC RADIOPHARMACEUTICALS",
        "DIAGNOSTIC DRUGS - IN VIVO OTHER": "DIAGNOSTIC DRUGS",
        "ANTIHYPERGLYCEMIC - SULFONYLUREA DERIVATIVES": "SULFONYLUREAS",
        "ANTINEOPLASTIC - ANTIMETABOLITE - FOLIC ACID ANALOGS": "ANTIMETABOLITES",
        "AMINOPENICILLIN ANTIBIOTIC": "AMINOPENICILLINS",
        "OPHTHALMIC - BETA BLOCKERS-CARBONIC ANHYDRASE INHIBITOR COMBINATIONS": "CARBONIC ANHYDRASE INHIBITORS",
        "CARBAPENEM ANTIBIOTICS (THIENAMYCINS)": "CARBAPENEMS",
        "ANTACID - BICARBONATE": "ANTACIDS - BICARBONATE",
        "DERMATOLOGICAL - EMOLLIENT MIXTURES": "EMOLLIENTS",
        "ANTINEOPLASTIC ANTIBIOTIC - ANTHRACYCLINES": "ANTINEOPLASTIC ANTIBIOTICS",
        "PROSTATIC HYPERTROPHY AGENT - ALPHA-1-ADRENOCEPTOR ANTAGONISTS": "PROSTATIC HYPERTROPHY AGENTS",
        "ANTIDIURETIC AND VASOPRESSOR HORMONES": "VASOPRESSORS",
        "LAXATIVE - LUBRICANT": "LUBRICANT LAXATIVES",
        "SODIUM CHLORIDE, PARENTERAL": "SODIUM",
        "THYROID HORMONES - SYNTHETIC T4 (THYROXINE)": "THYROID HORMONES",
        "THYROID HORMONES - SYNTHETIC T3 (TRIIODOTHYRONINE)": "THYROID HORMONES",
        "ANTIPSYCHOTIC - PHENOTHIAZINES, PIPERAZINE": "PHENOTHIAZINES",
        "MINERALS AND ELECTROLYTES - MAGNESIUM": "MAGNESIUM",
        "MUSCULOSKELETAL THERAPY AGENT - VISCOSUPPLEMENTS": "VISCOSUPPLEMENTS",
        "ANTINEOPLASTIC - ANTIMETABOLITE - UREA DERIVATIVES": "ANTIMETABOLITES",
        "TETRACYCLINE ANTIBIOTICS": "TETRACYCLINES",
        "GI ANTISPASMODIC - BELLADONNA ALKALOIDS": "ANTISPASMODICS",
        "ANTIPSYCHOTIC - ATYPICAL DOPAMINE-SEROTONIN ANTAG- BENZISOXAZOLE DERIV": "BENZISOXAZOLES",
        "LINCOSAMIDE ANTIBIOTICS": "LINCOSAMIDES",
        "ASTHMA/COPD -  PHOSPHODIESTERASE-4 (PDE4) INHIBITORS": "PHOSPHODIESTERASE 4 (PDE4) INHIBITORS",
        "GASTRIC ACID SECRETION REDUCING AGENTS - PROTON PUMP INHIBITORS (PPIS)": "PROTON PUMP INHIBITORS",
        "ANTIDIARRHEAL - ANTIPERISTALTIC AGENTS": "ANTIPERISTALTIC AGENTS",
        "B-COMPLEX VITAMIN COMBINATIONS": "B-COMPLEX VITAMINS",
        "ANTIRETROVIRAL-NUCLEOSIDE ANALOGS AND INTEGRASE INHIBITOR COMBINATIONS": "ANTIRETROVIRALS",
        "ANTIEMETIC - PHENOTHIAZINES": "PHENOTHIAZINES",
        "OPIOID ANTITUSSIVE-EXPECTORANT COMBINATIONS": "ANTITUSSIVES",
        "ANTIHYPERLIPIDEMIC - HMG COA REDUCTASE INHIBITORS (STATINS)": "HMG COA REDUCTASE INHIBITORS",
        "ANTIDEPRESSANT - SELECTIVE SEROTONIN REUPTAKE INHIBITORS (SSRIS)": "SELECTIVE SEROTONIN REUPTAKE INHIBITORS (SSRIS)",
        "ANTIHYPERLIPIDEMIC - BILE ACID SEQUESTRANTS": "BILE ACID SEQUESTRANTS",
        "SKELETAL MUSCLE RELAXANT - CENTRAL MUSCLE RELAXANTS": "CENTRAL MUSCLE RELAXANTS",
        "SEDATIVE-HYPNOTIC - BENZODIAZEPINES": "BENZODIAZEPINES",
        "ANTIRETROVIRAL - NON-NUCLEOSIDE REVERSE TRANSCRIPTASE INHIB (NNRTI)": "ANTIRETROVIRALS",
        "MULTIVITAMIN AND MINERAL COMBINATIONS": "MINERAL COMBINATIONS",
        "GLYCOPEPTIDE ANTIBIOTICS": "GLYCOPEPTIDES",
        "DERMATOLOGICAL - EMOLLIENTS": "EMOLLIENTS",
        "MIGRAINE THERAPY - SELECTIVE SEROTONIN AGONISTS 5-HT(1)": "SEROTONIN AGONISTS",
        "MINERALS AND ELECTROLYTES - BICARBONATE PRODUCING OR CONTAINING AGENTS": "BICARBONATES",
        "LOCAL ANESTHETIC - AMIDES": "LOCAL ANESTHETICS - AMIDES",
        "AMINOPENICILLIN ANTIBIOTIC - BETA-LACTAMASE INHIBITOR COMBINATIONS": "AMINOPENICILLINS",
        "URINARY ANTISPASMODIC - ANTICHOL., M(3) MUSCARINIC SELECTIVE (BLADDER)": "ANTISPASMODICS",
        "DERMATOLOGICAL - ANTIFUNGAL-GLUCOCORTICOID COMBINATIONS": "ANTIFUNGALS",
        "PULMONARY ARTERIAL HYPERTENSION - ENDOTHELIN RECEPTOR ANTAGONISTS": "PULMONARY HYPERTENSION - ENDOTHELIN RECEPTOR ANTAGONISTS",
        "MINERALS AND ELECTROLYTES - ZINC": "ZINC",
        "INSULIN ANALOGS - LONG ACTING": "INSULIN",
        "LOCAL ANESTHETIC - SYMPATHOMIMETIC COMBINATIONS": "SYMPATHOMIMETICS",
        "ANTIRETROVIRAL-NUCLEOSIDE, NUCLEOTIDE ANALOGS AND NON-NUCLEOSIDE RTI": "ANTIRETROVIRALS",
        "OPHTHALMIC - CARBONIC ANHYDRASE INHIBITORS": "CARBONIC ANHYDRASE INHIBITORS",
        "ANTIFUNGAL - AMPHOTERIC POLYENE MACROLIDES": "ANTIFUNGALS",
        "OPHTHALMIC - LOCAL ANESTHETIC ESTERS": "OPHTHALMIC LOCAL ANESTHETICS",
        "HUMAN INSULINS - SHORT ACTING": "INSULIN",
        "ANTIHYPERGLYCEMIC - MEGLITINIDE ANALOGS": "MEGLITINIDE ANALOGUES",
        "DIRECT ACTING VASODILATORS": "VASODILATORS",
        "LAXATIVE - STIMULANT": "STIMULANT LAXATIVES",
        "SALICYLATE ANALGESICS": "SALICYLATES",
        "DIAGNOSTIC DRUGS - CARDIOVASCULAR": "DIAGNOSTIC DRUGS",
    }

    missing_mapping = []

    for i, reference in enumerate(left):

        comparators = []
        sorted_results = []
        partial_results = []
        single_results =   []

        if reference in mapping.keys():
            print('Already have mapping: ', reference, ' -> ',mapping[reference])
            print()
            continue
        if reference in manual_mapping.keys():
            print('Overrided with manual mapping: ', reference,' -> ', manual_mapping[reference])
            print()
            continue
        

        for comparator in right:
            comparators.append(comparator)
            sorted_results.append(fuzz.token_sort_ratio(reference,comparator))
            partial_results.append(fuzz.partial_ratio(reference,comparator))
            single_results.append(fuzz.ratio(reference,comparator))
        
        sorted_distances = pd.DataFrame({'comparator': comparators, 'similarity': sorted_results}).sort_values(by=['similarity'],ascending=False).reset_index()
        partial_distances = pd.DataFrame({'comparator': comparators, 'similarity': partial_results}).sort_values(by=['similarity'],ascending=False).reset_index()
        single_distances = pd.DataFrame({'comparator': comparators, 'similarity': single_results}).sort_values(by=['similarity'],ascending=False).reset_index()

        if reference in exceptions:
            print('Exception')
            print('Top candidates: ', '\n', sorted_distances['comparator'][0], sorted_distances['similarity'][0], '\n',
                  partial_distances['comparator'][0], partial_distances['similarity'][0], '\n',
                  single_distances['comparator'][0], single_distances['similarity'][0])
            missing_mapping.append(reference)
            continue


        print(f'Comparing {reference}')
        if sorted_distances['similarity'][0] >= 90:
            print('Found similar: ', sorted_distances['comparator'][0], sorted_distances['similarity'][0])
            mapping[reference] = sorted_distances['comparator'][0]
        elif single_distances['similarity'][0] >= 90:
            diff = difflib.ndiff(reference, single_distances['comparator'][0])
            
            if not any(c.isnumeric() for c in diff):
                if not any(c in ['I','N'] for c in diff):
                    mapping[reference] = single_distances['comparator'][0]
                    print('Found similar: ', single_distances['comparator'][0], single_distances['similarity'][0])
            else:
                missing_mapping.append(reference)
                print('Reject: ', single_distances['comparator'][0], single_distances['similarity'][0])
        elif partial_distances['similarity'][0] >= 90:
            diff = difflib.ndiff(reference, partial_distances['comparator'][0])

            if not any(c.isnumeric() for c in diff):
                mapping[reference] = partial_distances['comparator'][0]
                print('Found similar: ', partial_distances['comparator'][0], partial_distances['similarity'][0])
        else:
            print('No similarity')
            print('Top candidates: ', '\n', sorted_distances['comparator'][0], sorted_distances['similarity'][0], '\n',
                  partial_distances['comparator'][0], partial_distances['similarity'][0], '\n',
                  single_distances['comparator'][0], single_distances['similarity'][0])
            missing_mapping.append(reference)

        print()

    print("Manually mapped: ", len(manual_mapping))
    print("Automatically mapped: ", len(mapping))
    print("Missing: ", len(missing_mapping))

    mapping = mapping | manual_mapping
    return mapping

only_cedars_meds = (set(cedars_meds['PHARM_SUBCLASS'].unique())).difference(set(ucla_meds['PHARM_SUBCLASS'].unique()))
all_ucla_meds = set(ucla_meds['PHARM_SUBCLASS'].unique())

mapping = create_mapping_dict(only_cedars_meds, all_ucla_meds)

In [ ]:
print("Total Cedars Rows: ", len(cedars_meds))
print("Total Cedars Rows Remapped: ", len(cedars_meds[cedars_meds['PHARM_SUBCLASS'].isin(mapping.keys())]))

print("Total UCLA Rows: ", len(ucla_meds))
print("Total UCLA Rows Corresponding to Mapping: ", len(ucla_meds[ucla_meds['PHARM_SUBCLASS'].isin(mapping.values())]))

In [ ]:
cedars_meds["PHARM_SUBCLASS"] = cedars_meds["PHARM_SUBCLASS"].replace(
    mapping
    )

assess_value_alignment(cedars_meds, ucla_meds, cols=['PHARM_SUBCLASS'], len_only=True)

In [ ]:
print(cedars_meds['PHARM_SUBCLASS'].value_counts())
print(ucla_meds['PHARM_SUBCLASS'].value_counts())

### Labs

In [ ]:
cedars_labs = data.load.read_files_and_combine(['Labs.txt'], args.cedars_crrt_data_dir)
ucla_labs = data.load.read_files_and_combine(['Labs.txt'], args.ucla_crrt_data_dir)

cedars_labs = data.longitudinal_features.map_encounter_to_patient(args.cedars_crrt_data_dir, cedars_labs)
# Cedars alignment. Assume contact date is diagnosis date since that's the date we have
cedars_labs = cedars_labs.rename({"RESULT": "RESULTS", 
                              'NAME': 'COMPONENT_NAME'}, axis=1)
ucla_labs = ucla_labs.rename({"RESULT": "RESULTS", 
                              'NAME': 'COMPONENT_NAME'}, axis=1)

assess_value_alignment(cedars_labs, ucla_labs, cols=['COMPONENT_NAME'], len_only=True)

In [ ]:
ucla_labs_temp = ucla_labs.copy()
cedars_labs_temp = cedars_labs.copy()

In [ ]:
left = (set(cedars_labs_temp['COMPONENT_NAME'].unique())).difference(set(ucla_labs_temp['COMPONENT_NAME'].unique()))
right = set(ucla_labs_temp['COMPONENT_NAME'].unique())
distance = jelly_pairwise_diff(left,right)
print(distance)
single_distance = distance[distance['Distance'] == 1]
print(single_distance)

In [ ]:
left = (set(cedars_labs_temp['COMPONENT_NAME'].unique())).difference(set(ucla_labs_temp['COMPONENT_NAME'].unique()))
right = set(ucla_labs_temp['COMPONENT_NAME'].unique())
distsort = fuzzy_pairwise_diff(left, right, mode='sort')
print(distsort)

##### Notes
* as opposed to medications, there are many more labs, so it is harder to manually inspect

* It seems like there are single character differences (e.g., hyphen or comma)

* There are also cases where the words in a string are the same but reordered

* We only use these two rules to match labs

* Below, we apply regex to remove the non-alphanumeric characters. Then we look for single character differences that are either an empty space or and 'S' (plurality). We then get the 'sorted' differences that look for the same strings but in different word orders

In [ ]:
import re

ucla_labs_temp['PSEUDO_COMPONENT_NAME'] = ucla_labs_temp['COMPONENT_NAME'].str.replace('[^a-zA-Z0-9]', ' ', regex=True)
ucla_labs_temp['PSEUDO_COMPONENT_NAME'] = ucla_labs_temp['PSEUDO_COMPONENT_NAME'].apply(lambda x: re.sub(' +', ' ', x).strip())

cedars_labs_temp['PSEUDO_COMPONENT_NAME'] = cedars_labs_temp['COMPONENT_NAME'].str.replace('[^a-zA-Z0-9]', ' ', regex=True)
cedars_labs_temp['PSEUDO_COMPONENT_NAME'] = cedars_labs_temp['PSEUDO_COMPONENT_NAME'].apply(lambda x: re.sub(' +', ' ', x).strip())

left = (set(cedars_labs_temp['PSEUDO_COMPONENT_NAME'].unique())).difference(set(ucla_labs_temp['PSEUDO_COMPONENT_NAME'].unique()))
right = set(ucla_labs_temp['PSEUDO_COMPONENT_NAME'].unique())
distance = jelly_pairwise_diff(left,right)
single_distance = distance[distance['Distance'] == 1]

cedars_mapping = []
ucla_mapping = []
for i, row in single_distance.iterrows():

    characters_left = dict.fromkeys(row.name[0],0)
    characters_right = dict.fromkeys(row.name[1],0)

    diff_chars = set(characters_left.keys()).symmetric_difference(set(characters_right.keys()))
    if len(diff_chars) > 0:
        if 'S' in diff_chars or ' ' in diff_chars:
            cedars_mapping.append(row.name[0])
            ucla_mapping.append(row.name[1])
    else:

        for character in characters_left.keys():
            characters_left[character] = row.name[0].count(character)

            characters_right[character] = row.name[1].count(character)
        
            if characters_left[character] != characters_right[character]:
                if character == 'S' or character == ' ':
                    cedars_mapping.append(row.name[0])
                    ucla_mapping.append(row.name[1])                                                       
print(len(cedars_mapping))


left = (set(cedars_labs_temp['PSEUDO_COMPONENT_NAME'].unique())).difference(set(ucla_labs_temp['PSEUDO_COMPONENT_NAME'].unique()))
right = set(ucla_labs_temp['PSEUDO_COMPONENT_NAME'].unique())
distsort = fuzzy_pairwise_diff(left, right, mode='sort')

resorted = distsort[distsort['Distance'] >= 97]
for i, row in resorted.iterrows():
    if 'FACTOR' not in row.name[1]:
        cedars_mapping.append(row.name[0])
        ucla_mapping.append(row.name[1])
print(len(cedars_mapping))

resorted = distsort[distsort['Distance'] == 96]
for i, row in resorted.iterrows():
    if 'ESTIMATE' in row.name[1]:
        cedars_mapping.append(row.name[0])
        ucla_mapping.append(row.name[1])

print(len(cedars_mapping))
same = set(cedars_labs_temp['PSEUDO_COMPONENT_NAME']).intersection(ucla_labs_temp['PSEUDO_COMPONENT_NAME'])
same = list(same)

cedars_mapping = cedars_mapping + same
ucla_mapping = ucla_mapping + same
print(len(cedars_mapping))

cedars_mapping_df = cedars_labs_temp[cedars_labs_temp['PSEUDO_COMPONENT_NAME'].isin(cedars_mapping)]
ucla_mapping_df = ucla_labs_temp[ucla_labs_temp['PSEUDO_COMPONENT_NAME'].isin(ucla_mapping)]

cedars_lab_mapping = dict(
    zip(cedars_mapping_df["COMPONENT_NAME"], cedars_mapping_df["PSEUDO_COMPONENT_NAME"])
)

pseudo_lab_mapping = dict(
    zip(cedars_mapping, ucla_mapping)
)

ucla_lab_mapping= dict(
    zip(ucla_mapping_df["PSEUDO_COMPONENT_NAME"], ucla_mapping_df["COMPONENT_NAME"])
)

final_mapping = {}

for key in cedars_lab_mapping:
    final_mapping[key] = ucla_lab_mapping[pseudo_lab_mapping[cedars_lab_mapping[key]]]
print(len(final_mapping))

In [ ]:
print('Total Cedars Rows: ', len(cedars_labs_temp))
print('Total Cedars Rows Remapped: ', len(cedars_labs_temp[cedars_labs_temp['COMPONENT_NAME'].isin(final_mapping.keys())]))

print('Total UCLA Rows: ', len(ucla_labs_temp))
print('Total UCLA Rows Corresponding to Mapping: ', len(ucla_labs_temp[ucla_labs_temp['COMPONENT_NAME'].isin(final_mapping.values())]))

In [ ]:
cedars_labs_temp["COMPONENT_NAME"] = cedars_labs_temp["COMPONENT_NAME"].replace(
    final_mapping
    )

assess_value_alignment(cedars_labs_temp, ucla_labs_temp, cols=['COMPONENT_NAME'], len_only=True)

In [ ]:
print(cedars_meds['PHARM_SUBCLASS'].value_counts())
print(ucla_meds['PHARM_SUBCLASS'].value_counts())

# Descriptive Report

In [ ]:
import data.load

preprocessed_df = data.load.load_data(args, "cedars_crrt")


In [ ]:
preprocessed_df

In [ ]:
import upsetplot
from matplotlib import cm
import matplotlib.pyplot as plt
import pandas as pd
def plot_upsetplot(preprocessed_df: pd.DataFrame, cohort: str, colnames: list[str]):
    fig = plt.figure(figsize=(10, 30))
    indicator_cols = preprocessed_df[colnames].astype(bool)
    data = pd.concat([indicator_cols, preprocessed_df["recommend_crrt"]],axis=1).replace({0: "Do not recommend CRRT", 1: "Recommend CRRT"})
    myplot = upsetplot.UpSet(data.set_index(colnames), intersection_plot_elements=0, show_counts=True, show_percentages=True, element_size=50)
    myplot.add_stacked_bars(by="recommend_crrt", colors=cm.Set2, elements=10)
    myplot.plot()
    plt.title(f"{cohort} Patient Type Breakdown")
    plt.show()

In [ ]:
race_cols = preprocessed_df.columns[preprocessed_df.columns.str.contains("RACE_")].to_list()
race_cols

In [ ]:
disease_groups =  [f"{name}_pt_indicator" for name in ["heart", "liver", "infection"]]
for columns in [disease_groups, race_cols]:
    plot_upsetplot(preprocessed_df, "CEDARS CRRT", columns)


In [ ]:
print(preprocessed_df["recommend_crrt"].value_counts())
print(preprocessed_df["recommend_crrt"].value_counts(normalize=True)*100)

In [ ]:
tmp_df = preprocessed_df.drop(preprocessed_df.columns[preprocessed_df.columns.str.contains("^Unnamed")], axis=1)
def print_missing_info(filter = None):
    if filter is not None:
        total_notmissing = sum(~tmp_df[filter].isna().any(axis=1))
    else:
        total_notmissing = sum(~tmp_df.isna().any(axis=1))
    print(f"Number of patients not missing any data: {total_notmissing}, ({total_notmissing/tmp_df.shape[0] * 100}%)")
print("All")
print_missing_info()

In [ ]:
(tmp_df.drop(["infection_pt_indicator", "liver_pt_indicator", "heart_pt_indicator", "recommend_crrt", "CRRT Year"], axis=1).isna().mean() * 100).sort_values()

amount_missing = tmp_df.drop("recommend_crrt",axis=1).isna().mean().sort_values()
amount_missing.name = "% Missing"
amount_missing.index.name = "Variables"
(amount_missing*100).to_csv(join("C:/Users/jeffe/OneDrive - UCLA IT Services/UCLA/2023_Winter/Rotation/Data/Cedars", "variables_and_amount_missing.csv"))
amount_missing

In [ ]:
tmp_df[tmp_df.columns[tmp_df.columns.str.contains("_na")]].mean()

# Packages

In [ ]:
fuzzywuzzy
jellyfish

# Load for Training/Eval

In [ ]:
%reload_ext autoreload
%autoreload 2

import sys
import os
from os import getcwd
from os.path import join
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
sys.path.insert(0, join(getcwd(), "../module_code"))

import data.load
import cli_utils 

sys.argv = [sys.argv[0]]
cli_utils.load_cli_args("../options.yml")
args = cli_utils.init_cli_args()

In [ ]:
from data.sklearn_loaders import SklearnCRRTDataModule
from exp.static_learning import generate_filters

data = SklearnCRRTDataModule.from_argparse_args(args, filters=generate_filters())

In [ ]:
reference_ids = None
original_columns = None
data_transform = None
data.setup(
    args,
    reference_ids=reference_ids,
    reference_cols=original_columns,
    data_transform=data_transform,
)


In [ ]:
print(data.train[0].shape)
# data.val[0].shape
print(data.test[0].shape)